<a href="https://colab.research.google.com/github/Mariam-Elbishbeashy/ANA_AI_Models/blob/main/Questionnaire-models%20%26%20datasets/Questionnaire_arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
from collections import Counter

# ==================== تعريف الشخصيات بالعربي المصري ====================
CHARACTERS_AR = [
    # 🛡️ المديرين (الحماة الاستباقيين)
    "الناقد الداخلي", "الكمالي", "المُرضي للناس", "المتحكّم",
    "الجزء البارد", "مدمن الشغل", "الجزء الحيران",

    # 🔥 رجال الإطفاء (الحماة التفاعليين)
    "المؤجِّل", "الآكل/النهم", "اللاعب المفرط",

    # 😥 المنفيين (الأجزاء المجروحة)
    "الجزء الوحيد", "الجزء الخايف", "الجزء المهمَل", "الجزء الخجول",
    "الجزء المرهَق", "الجزء المعتمد", "الجزء الغيور", "الطفل الجريح"
]

# مجموعات الأنماط
MANAGERS_AR = ["الناقد الداخلي", "الكمالي", "المُرضي للناس", "المتحكّم",
               "الجزء البارد", "مدمن الشغل", "الجزء الحيران"]

FIREFIGHTERS_AR = ["المؤجِّل", "الآكل/النهم", "اللاعب المفرط"]

EXILES_AR = ["الجزء الوحيد", "الجزء الخايف", "الجزء المهمَل", "الجزء الخجول",
             "الجزء المرهَق", "الجزء المعتمد", "الجزء الغيور", "الطفل الجريح"]

# ==================== تعيينات الأسئلة بالعربي المصري ====================
QUESTION_MAPPINGS_AR = {
    'Q1': {
        0: ["المتحكّم", "الكمالي"],               # عمل قوائم وخطط
        1: ["مدمن الشغل", "الجزء البارد"],       # الضغط والإنجاز
        2: ["المُرضي للناس", "الجزء المعتمد"],   # التأكد من الآخرين
        3: ["المؤجِّل", "اللاعب المفرط"],        # البحث عن إلهاء
        4: ["الجزء البارد", "المتحكّم"],         # "أنا هقدر أتعامل مع الموضوع"
        5: ["الجزء الحيران", "الجزء الوحيد"]     # وقت هادي لوحدي
    },

    'Q2_slider': {
        '0-20%': [],  # بسيط
        '21-50%': ["الكمالي"],  # بعض الصفات
        '51-80%': ["الكمالي", "الناقد الداخلي"],  # نشيط
        '81-100%': ["الكمالي", "الناقد الداخلي", "مدمن الشغل"]  # مسيطر
    },

    'Q3': {
        0: ["المتحكّم", "الكمالي"],               # الإصلاح الفوري
        1: ["الآكل/النهم", "المؤجِّل"],           # الإلهاء
        2: ["الطفل الجريح", "الجزء الخايف"],      # الشعور بالصغر
        3: ["الناقد الداخلي", "الجزء البارد"],    # "مينفعش أحس كده"
        4: ["الجزء الحيران", "الجزء المهمَل"],    # الانفصال
        5: ["الناقد الداخلي", "الجزء الخجول"]     # الغضب من النفس
    },

    'Q4_slider': {
        '0-20%': [],  # بسيط
        '21-50%': ["الجزء الوحيد"],  # بعض الصفات
        '51-80%': ["الجزء الوحيد", "الجزء المعتمد", "الطفل الجريح"],  # نشيط
        '81-100%': ["الجزء الوحيد", "الجزء المهمَل", "الطفل الجريح"]  # مسيطر
    },

    'Q5': {
        0: ["الكمالي", "الناقد الداخلي"],        # الجزء الكمالي
        1: ["الجزء الوحيد", "الجزء المهمَل"],    # الجزء الوحيد
        2: ["المتحكّم", "الجزء الخايف"],         # الجزء المتحكم
        3: ["المؤجِّل", "اللاعب المفرط"],        # الجزء الهارب
        4: ["الجزء الخجول", "الناقد الداخلي"],   # الجزء الخجول
        5: ["الجزء الحيران", "الطفل الجريح"]     # الجزء الحيران
    },

    'Q6': {
        0: ["الناقد الداخلي", "الكمالي"],        # "مش مشكلة انك تغلط"
        1: ["الجزء الخايف", "الجزء المعتمد"],   # "أنا مش هسيبك لوحدك"
        2: ["المُرضي للناس", "الجزء الخجول"],    # "مش محتاج تحاول علشان تاخد حب"
        3: ["الجزء المهمَل", "الجزء البارد"],    # "مشاعرك ليها قيمة"
        4: ["الجزء الخايف", "الجزء المرهَق"],    # "أنت في أمان دلوقتي"
        5: ["الجزء الخجول", "الطفل الجريح"]      # "أنا بقبل كل حاجة فيك"
    },

    'Q7': {
        0: ["مدمن الشغل", "المتحكّم"],           # العمل أكثر
        1: ["الآكل/النهم", "المؤجِّل"],           # الطعام المريح/المسلسلات
        2: ["الجزء البارد", "الجزء المهمَل"],    # الاستسلام النفسي
        3: ["المُرضي للناس", "الجزء المعتمد"],   # إرضاء الجميع
        4: ["اللاعب المفرط", "المؤجِّل"],        # ألعاب الفيديو
        5: ["الناقد الداخلي", "الجزء الخجول"]    # انتقاد النفس
    },

    'Q8_slider': {
        '0-20%': [],  # نشاط إطفائي بسيط
        '21-50%': ["المؤجِّل", "الآكل/النهم"],  # بعض الصفات
        '51-80%': ["المؤجِّل", "الآكل/النهم", "اللاعب المفرط"],  # أنماط نشيطة
        '81-100%': ["المؤجِّل", "الآكل/النهم", "اللاعب المفرط"]  # استراتيجية مسيطرة
    },

    'Q9': {
        0: ["المُرضي للناس", "الجزء المعتمد"],   # وضع الحدود
        1: ["الجزء الخايف", "الجزء الوحيد"],     # الثقة في اهتمام الآخرين
        2: ["الجزء الغيور", "الجزء الخجول"],     # عدم المقارنة
        3: ["الجزء المعتمد", "الجزء المهمَل"],   # طلب الاحتياجات
        4: ["الجزء الوحيد", "الطفل الجريح"],     # الشعور بالمشاهدة والفهم
        5: ["الجزء المعتمد", "الجزء الخايف"]     # الاعتماد على أحد
    },

    'Q10': {
        0: ["المُرضي للناس", "الجزء المعتمد"],   # "لازم يعجبوا بيك"
        1: ["الجزء البارد", "المتحكّم"],         # "ماتورّيش أي نقطة ضعف"
        2: ["الجزء الغيور", "الجزء الخجول"],     # "هما متحكمين في حياتهم أكتر مني"
        3: ["الجزء الغيور", "الناقد الداخلي"],   # "عايز أبقى زيهم"
        4: ["الجزء الوحيد", "الجزء الحيران"],    # "أنا مش منتمي ليهم هنا"
        5: ["الجزء الخجول", "الجزء الخايف"]      # "لو شافوا حقيقتي هيعملوا إيه؟"
    },

    'Q11': {
        0: ["الناقد الداخلي", "الجزء الخجول"],   # "مش كفاية"
        1: ["الجزء الوحيد", "الجزء المهمَل"],    # وحدة عميقة
        2: ["الجزء الخايف", "الكمالي"],          # قلق من الفشل
        3: ["الجزء المهمَل", "الجزء الحيران"],   # خدر نفسي أو فراغ
        4: ["الجزء الخجول", "مدمن الشغل"],       # ذنب
        5: ["الجزء الحيران", "الطفل الجريح"]     # حيرة
    },

    'Q12': {
        0: ["الكمالي", "مدمن الشغل"],            # مدى المحاولة
        1: ["الجزء المهمَل", "الطفل الجريح"],    # مدى الألم
        2: ["الجزء الخايف", "الكمالي"],          # الخوف من الفشل
        3: ["الجزء الوحيد", "الجزء المهمَل"],    # قوة الوحدة
        4: ["الجزء المعتمد", "الجزء المرهَق"],   # الحاجة للمساعدة
        5: ["الجزء الحيران", "الطفل الجريح"]     # مدى الحيرة
    },

    'Q13': {
        0: ["الجزء المرهَق", "الجزء الخايف"],    # الشعور بالإغراق
        1: ["الجزء الغيور", "الجزء الخجول"],     # المقارنة بالآخرين
        2: ["الكمالي", "المتحكّم"],               # التركيز على الأخطاء
        3: ["الجزء المعتمد", "الجزء الخجول"],    # الشعور بالحمولة
        4: ["اللاعب المفرط", "الجزء الحيران"],   # الهروب للعالم الخيالي
        5: ["الناقد الداخلي", "الجزء البارد"],   # الغضب من النفس
        6: ["الجزء المعتمد", "الطفل الجريح"],    # الرغبة في الرعاية
        7: ["الجزء المهمَل", "الجزء البارد"]      # الخدر وعدم الإحساس
    }
}

# أنماط قوية لكل شخصية (مؤشرات رئيسية)
CHARACTER_SIGNATURES_AR = {
    "الكمالي": ["Q2:high", "Q1:0", "Q3:0", "Q5:0"],
    "الناقد الداخلي": ["Q2:high", "Q3:3", "Q5:0", "Q11:0"],
    "المُرضي للناس": ["Q10:0", "Q7:3", "Q9:0", "Q1:2"],
    "المتحكّم": ["Q1:0", "Q3:0", "Q10:1", "Q7:0"],
    "الجزء البارد": ["Q1:4", "Q3:3", "Q7:2", "Q13:7"],
    "مدمن الشغل": ["Q1:1", "Q7:0", "Q12:0", "Q2:high"],
    "الجزء الحيران": ["Q1:5", "Q3:4", "Q5:5", "Q11:5"],
    "المؤجِّل": ["Q8:high", "Q1:3", "Q3:1", "Q7:4"],
    "الآكل/النهم": ["Q8:high", "Q3:1", "Q7:1", "Q13:1"],
    "اللاعب المفرط": ["Q8:high", "Q1:3", "Q7:4", "Q13:4"],
    "الجزء الوحيد": ["Q4:high", "Q1:5", "Q11:1", "Q12:3"],
    "الجزء الخايف": ["Q11:2", "Q6:1", "Q9:1", "Q13:0"],
    "الجزء المهمَل": ["Q4:high", "Q3:4", "Q6:3", "Q11:3"],
    "الجزء الخجول": ["Q11:0", "Q6:5", "Q9:2", "Q10:5"],
    "الجزء المرهَق": ["Q13:0", "Q12:4", "Q6:4", "Q11:2"],
    "الجزء المعتمد": ["Q9:5", "Q12:4", "Q10:0", "Q6:1"],
    "الجزء الغيور": ["Q10:2", "Q9:2", "Q13:1", "Q10:3"],
    "الطفل الجريح": ["Q6:5", "Q12:1", "Q3:2", "Q9:4"]
}

def generate_probabilistic_dataset_ar(num_samples=75000):
    """إنشاء مجموعة بيانات بالعربي مع احتمالية وجود الشخصيات"""

    print(f"جارٍ إنشاء {num_samples:,} عينة عربية احتمالية...")

    dataset = []
    slider_values = ['0-20%', '21-50%', '51-80%', '81-100%']

    for i in range(num_samples):
        if i % 10000 == 0 and i > 0:
            print(f"  تم إنشاء {i:,} عينة...")

        responses = {}

        # ===== 1. توليد الشخصية الأساسية (الجوهرية) =====
        primary_char = random.choice(CHARACTERS_AR)
        primary_archetype = None
        if primary_char in MANAGERS_AR:
            primary_archetype = "مدير"
        elif primary_char in FIREFIGHTERS_AR:
            primary_archetype = "إطفائي"
        else:
            primary_archetype = "منفي"

        # ===== 2. توليد الشخصيات الثانوية =====
        secondary_chars = []
        tertiary_chars = []

        # بناءً على الشخصية الأساسية، الشخصيات الثانوية المحتملة
        if primary_archetype == "مدير":
            # المديرون غالبًا ما يعملون مع مديرين آخرين أو لديهم أجزاء منفية
            candidate_secondaries = [c for c in CHARACTERS_AR if c != primary_char]
            # تفضيل نفس النمط أو الأجزاء المنفية ذات الصلة
            secondary = random.choice([c for c in candidate_secondaries if
                                      (c in MANAGERS_AR) or
                                      (primary_char in ["الناقد الداخلي", "الكمالي"] and c in ["الجزء الخجول", "الطفل الجريح"])])
            secondary_chars.append(secondary)

            # الثالثية: غالبًا جزء منفي
            remaining = [c for c in EXILES_AR if c not in secondary_chars]
            if remaining:
                tertiary_chars.append(random.choice(remaining))

        elif primary_archetype == "إطفائي":
            # رجال الإطفاء غالبًا ما يعملون معًا أو لديهم مديرين يحفزونهم
            candidate_secondaries = [c for c in FIREFIGHTERS_AR if c != primary_char]
            if candidate_secondaries:
                secondary_chars.append(random.choice(candidate_secondaries))

            # غالبًا لديهم مدير صارم جدًا
            tertiary_chars.append(random.choice(["الناقد الداخلي", "الكمالي", "المتحكّم"]))

        else:  # منفي
            # المنفيون غالبًا يأتون مع حماة (مديرين/إطفائيين)
            if primary_char in ["الجزء الوحيد", "الجزء المهمَل"]:
                secondary_chars.append(random.choice(["المُرضي للناس", "مدمن الشغل"]))
                tertiary_chars.append(random.choice(["الجزء البارد", "الجزء الحيران"]))
            elif primary_char in ["الجزء الخجول", "الطفل الجريح"]:
                secondary_chars.append(random.choice(["الناقد الداخلي", "الكمالي"]))
                tertiary_chars.append("المؤجِّل")
            else:
                secondary_chars.append(random.choice(MANAGERS_AR + FIREFIGHTERS_AR))

        # التأكد من وجود 2-4 شخصيات إجمالاً
        all_chars = list(set([primary_char] + secondary_chars + tertiary_chars))
        if len(all_chars) > 4:
            all_chars = random.sample(all_chars, 4)
        elif len(all_chars) < 2:
            all_chars.append(random.choice([c for c in CHARACTERS_AR if c not in all_chars]))

        # ===== 3. توليد الإجابات بناءً على الشخصيات =====
        # توليد إجابات تعكس الشخصية الأساسية بقوة
        for q in ['Q1', 'Q3', 'Q5', 'Q6', 'Q7', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13']:
            # البدء بالإجابات المفضلة للشخصية الأساسية
            primary_answers = []
            for ans, chars in QUESTION_MAPPINGS_AR.get(q, {}).items():
                if primary_char in chars:
                    primary_answers.append(ans)

            # إضافة إجابات الشخصيات الثانوية
            secondary_answers = []
            for char in secondary_chars:
                for ans, chars in QUESTION_MAPPINGS_AR.get(q, {}).items():
                    if char in chars:
                        secondary_answers.append(ans)

            # الدمج مع أوزان: الأساسية 60٪، الثانوية 40٪
            all_answers = []
            if primary_answers:
                n_primary = random.randint(1, min(2, len(primary_answers)))
                all_answers.extend(random.sample(primary_answers, n_primary))

            if secondary_answers:
                n_secondary = random.randint(0, min(1, len(secondary_answers)))
                if n_secondary > 0:
                    all_answers.extend(random.sample(secondary_answers, n_secondary))

            # التأكد من وجود 1-3 إجابات إجمالاً
            if not all_answers:
                max_opt = 6 if q != 'Q13' else 8
                all_answers = random.sample(range(max_opt), random.randint(1, 2))

            if len(all_answers) > 3:
                all_answers = random.sample(all_answers, 3)

            responses[q] = ','.join(map(str, sorted(set(all_answers))))

        # توليد إجابات أشرطة التمرير
        def get_slider_for_char_ar(char, q_prefix):
            """الحصول على قيمة شريط التمرير بناءً على الشخصية"""
            if char == "الكمالي" and q_prefix == "Q2":
                return random.choices(['51-80%', '81-100%'], weights=[40, 60])[0]
            elif char == "الجزء الوحيد" and q_prefix == "Q4":
                return random.choices(['51-80%', '81-100%'], weights=[40, 60])[0]
            elif char in FIREFIGHTERS_AR and q_prefix == "Q8":
                return random.choices(['51-80%', '81-100%'], weights=[40, 60])[0]
            else:
                return random.choice(slider_values)

        # أشرطة التمرير تعكس أقوى شخصية لتلك السمة
        responses['Q2'] = get_slider_for_char_ar(primary_char, "Q2")
        responses['Q4'] = get_slider_for_char_ar(primary_char, "Q4")
        responses['Q8'] = get_slider_for_char_ar(primary_char, "Q8")

        # ===== 4. إنشاء أهداف الاحتمالية =====
        # الشخصية الأساسية: 80-100٪ احتمالية
        # الشخصيات الثانوية: 40-70٪ احتمالية
        # الشخصيات الثالثية: 20-50٪ احتمالية
        # الآخرون: 0-20٪ احتمالية

        for char in CHARACTERS_AR:
            col_name = f'prob_{char.replace(" ", "_").replace("/", "_")}'

            if char == primary_char:
                # الأساسية: احتمالية عالية
                responses[col_name] = round(random.uniform(0.8, 0.98), 2)
            elif char in secondary_chars:
                # الثانوية: احتمالية متوسطة-عالية
                responses[col_name] = round(random.uniform(0.4, 0.7), 2)
            elif char in tertiary_chars:
                # الثالثية: احتمالية متوسطة-منخفضة
                responses[col_name] = round(random.uniform(0.2, 0.5), 2)
            else:
                # الآخرون: احتمالية منخفضة، لكن ليست صفر
                responses[col_name] = round(random.uniform(0.0, 0.2), 2)

        # ===== 5. إضافة الميزات المشتقة =====
        # الميزة 1: عدد الإجابات لكل سؤال
        for q in ['Q1', 'Q3', 'Q5', 'Q6', 'Q7', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13']:
            responses[f'{q}_count'] = len(responses[q].split(','))

        # الميزة 2: تحويل أشرطة التمرير إلى أرقام
        slider_map = {'0-20%': 0.1, '21-50%': 0.35, '51-80%': 0.65, '81-100%': 0.9}
        responses['Q2_num'] = slider_map[responses['Q2']]
        responses['Q4_num'] = slider_map[responses['Q4']]
        responses['Q8_num'] = slider_map[responses['Q8']]

        # الميزة 3: مؤشرات الأنماط
        responses['has_manager_indicators'] = int(
            responses['Q2_num'] > 0.6 or  # الكمالية
            responses.get('Q1', '').split(',').count('0') > 0 or  # المتحكّم
            responses.get('Q10', '').split(',').count('0') > 0   # المُرضي للناس
        )

        responses['has_exile_indicators'] = int(
            responses['Q4_num'] > 0.6 or  # الوحدة
            responses.get('Q11', '').split(',').count('1') > 0 or  # شعور الوحدة
            responses.get('Q13', '').split(',').count('0') > 0    # الإرهاق
        )

        responses['has_firefighter_indicators'] = int(
            responses['Q8_num'] > 0.6 or  # الهروب
            responses.get('Q7', '').split(',').count('1') > 0 or  # الأكل المريح
            responses.get('Q7', '').split(',').count('4') > 0    # الألعاب
        )

        dataset.append(responses)

    # إنشاء DataFrame
    question_cols = [f'Q{i}' for i in range(1, 14)]
    derived_cols = [col for col in dataset[0].keys() if col.startswith('Q') and '_num' in col or '_count' in col or 'has_' in col]
    prob_cols = [f'prob_{char.replace(" ", "_").replace("/", "_")}' for char in CHARACTERS_AR]

    all_cols = question_cols + derived_cols + prob_cols
    df = pd.DataFrame(dataset)[all_cols]

    return df, CHARACTERS_AR

def create_top3_dataset_ar(df, characters):
    """تحويل مجموعة البيانات الاحتمالية إلى تنسيق أفضل 3 للتدريب"""

    prob_cols = [f'prob_{char.replace(" ", "_").replace("/", "_")}' for char in characters]
    prob_matrix = df[prob_cols].values

    # الحصول على أفضل 3 مؤشرات لكل صف
    top3_indices = np.argsort(prob_matrix, axis=1)[:, -3:][:, ::-1]

    # إنشاء مجموعة بيانات جديدة مع أفضل 3 شخصيات كسلاسل نصية
    top3_data = []
    for idx, row in df.iterrows():
        row_data = {col: row[col] for col in df.columns if not col.startswith('prob_')}

        # الحصول على أفضل 3 شخصيات واحتمالاتها
        top3_chars = []
        top3_probs = []

        for rank in range(3):
            char_idx = top3_indices[idx, rank]
            char_name = characters[char_idx]
            prob = prob_matrix[idx, char_idx]

            row_data[f'top{rank+1}_char'] = char_name
            row_data[f'top{rank+1}_prob'] = round(prob, 2)

            top3_chars.append(char_name)
            top3_probs.append(prob)

        # إضافة مقاييس الثقة
        row_data['confidence_score'] = round(np.mean(top3_probs), 2)
        row_data['margin_1_2'] = round(top3_probs[0] - top3_probs[1], 2)
        row_data['top3_list'] = ','.join(top3_chars)

        top3_data.append(row_data)

    top3_df = pd.DataFrame(top3_data)
    return top3_df

def analyze_top3_dataset_ar(df, characters):
    """تحليل مجموعة البيانات لأفضل 3"""

    print("\n" + "="*60)
    print("تحليل مجموعة البيانات العربية لأفضل 3 شخصيات")
    print("="*60)

    # ظهور الشخصيات في أفضل 3
    char_counts = {char: 0 for char in characters}

    for i in range(1, 4):
        col = f'top{i}_char'
        counts = df[col].value_counts()
        for char, count in counts.items():
            char_counts[char] += count

    print("ظهور الشخصيات في مراكز أفضل 3:")
    print("-" * 50)

    sorted_chars = sorted(char_counts.items(), key=lambda x: x[1], reverse=True)
    for char, count in sorted_chars:
        percentage = (count / (len(df) * 3)) * 100
        print(f"{char:25} {count:6,} ({percentage:5.1f}% من مراكز أفضل 3)")

    # تحليل الثقة
    print(f"\nإحصاءات الثقة:")
    print(f"متوسط الثقة: {df['confidence_score'].mean():.2f}")
    print(f"ثقة عالية (>0.7): {(df['confidence_score'] > 0.7).sum():,} عينة")
    print(f"ثقة متوسطة (0.4-0.7): {((df['confidence_score'] >= 0.4) & (df['confidence_score'] <= 0.7)).sum():,} عينة")
    print(f"ثقة منخفضة (<0.4): {(df['confidence_score'] < 0.4).sum():,} عينة")

    # تحليل الهامش
    print(f"\nالفرق بين الأول والثاني:")
    print(f"متوسط الفرق: {df['margin_1_2'].mean():.2f}")
    print(f"فائز واضح (فرق > 0.2): {(df['margin_1_2'] > 0.2).sum():,} عينة")
    print(f"منافسة قريبة (فرق < 0.1): {(df['margin_1_2'] < 0.1).sum():,} عينة")

    # توزيع الأنماط في المركز الأول
    top1_archetypes = []
    for char in df['top1_char']:
        if char in MANAGERS_AR:
            top1_archetypes.append("مدير")
        elif char in FIREFIGHTERS_AR:
            top1_archetypes.append("إطفائي")
        else:
            top1_archetypes.append("منفي")

    archetype_counts = pd.Series(top1_archetypes).value_counts()
    print(f"\nنمط الشخصية في المركز الأول:")
    for archetype, count in archetype_counts.items():
        print(f"  {archetype}: {count:,} عينة ({count/len(df):.1%})")

def prepare_model_training_ar(df):
    """تحضير الميزات والأهداف لتدريب النموذج"""

    # أعمدة الأسئلة
    question_cols = [f'Q{i}' for i in range(1, 14)]
    derived_cols = [col for col in df.columns if col.endswith('_num') or col.endswith('_count') or ('has_' in col and 'indicator' in col)]
    feature_cols = question_cols + derived_cols

    # أعمدة الأهداف (أفضل 3 شخصيات)
    target_cols = ['top1_char', 'top2_char', 'top3_char']

    print(f"\nإعداد التدريب:")
    print(f"الميزات: {len(feature_cols)} عمود")
    print(f"العينات: {len(df):,}")

    return df[feature_cols], df[target_cols], feature_cols

# ==================== إنشاء مجموعة البيانات العربية المحسنة ====================
print("="*60)
print("إنشاء مجموعة بيانات ANA العربية المحسنة للتنبؤ بأفضل 3 شخصيات")
print("="*60)

DATASET_SIZE = 100000  # مجموعة بيانات أكبر لتعلم أفضل
prob_df_ar, characters_ar = generate_probabilistic_dataset_ar(DATASET_SIZE)

# التحويل إلى تنسيق أفضل 3
top3_df_ar = create_top3_dataset_ar(prob_df_ar, characters_ar)

# حفظ مجموعات البيانات
prob_df_ar.to_csv('ana_dataset_probabilistic_ar.csv', index=False, encoding='utf-8-sig')
top3_df_ar.to_csv('ana_dataset_top3_ar.csv', index=False, encoding='utf-8-sig')

print(f"\n✅ تم حفظ مجموعة البيانات الاحتمالية العربية: 'ana_dataset_probabilistic_ar.csv'")
print(f"✅ تم حفظ مجموعة البيانات العربية لأفضل 3: 'ana_dataset_top3_ar.csv'")

# التحليل
analyze_top3_dataset_ar(top3_df_ar, characters_ar)

# التحضير لتدريب النموذج
X_ar, y_ar, feature_cols_ar = prepare_model_training_ar(top3_df_ar)

print(f"\n" + "="*60)
print("مجموعة البيانات العربية جاهزة لتدريب النموذج!")
print("="*60)

# عرض عينة من البيانات
print("\nعينة من مجموعة البيانات العربية:")
print("-" * 50)
sample = top3_df_ar.head(3)
for i, (_, row) in enumerate(sample.iterrows()):
    print(f"\nعينة {i+1}:")
    print(f"  الشخصية الأولى: {row['top1_char']} (احتمال: {row['top1_prob']})")
    print(f"  الشخصية الثانية: {row['top2_char']} (احتمال: {row['top2_prob']})")
    print(f"  الشخصية الثالثة: {row['top3_char']} (احتمال: {row['top3_prob']})")
    print(f"  درجة الثقة: {row['confidence_score']}")
    print(f"  الفرق بين الأول والثاني: {row['margin_1_2']}")

إنشاء مجموعة بيانات ANA العربية المحسنة للتنبؤ بأفضل 3 شخصيات
جارٍ إنشاء 100,000 عينة عربية احتمالية...
  تم إنشاء 10,000 عينة...
  تم إنشاء 20,000 عينة...
  تم إنشاء 30,000 عينة...
  تم إنشاء 40,000 عينة...
  تم إنشاء 50,000 عينة...
  تم إنشاء 60,000 عينة...
  تم إنشاء 70,000 عينة...
  تم إنشاء 80,000 عينة...
  تم إنشاء 90,000 عينة...

✅ تم حفظ مجموعة البيانات الاحتمالية العربية: 'ana_dataset_probabilistic_ar.csv'
✅ تم حفظ مجموعة البيانات العربية لأفضل 3: 'ana_dataset_top3_ar.csv'

تحليل مجموعة البيانات العربية لأفضل 3 شخصيات
ظهور الشخصيات في مراكز أفضل 3:
--------------------------------------------------
المؤجِّل                  25,923 (  8.6% من مراكز أفضل 3)
الكمالي                   25,796 (  8.6% من مراكز أفضل 3)
الناقد الداخلي            25,401 (  8.5% من مراكز أفضل 3)
الجزء الحيران             20,055 (  6.7% من مراكز أفضل 3)
المتحكّم                  19,990 (  6.7% من مراكز أفضل 3)
مدمن الشغل                19,950 (  6.7% من مراكز أفضل 3)
الجزء البارد              19,750 (  6

In [ ]:
import pandas as pd
import numpy as np
import pickle
import random
import time
from typing import Dict, List, Tuple, Any, Optional
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("🚀 النموذج الجاهز للاستخدام للتنبؤ بالشخصيات (العربية)")
print("="*80)

# ==================== 1. تحميل مجموعة البيانات العربية ====================
print("\n1. تحميل مجموعة البيانات العربية...")
df = pd.read_csv('ana_dataset_top3_ar.csv')
print(f"   ✅ تم تحميل مجموعة البيانات: {len(df):,} عينة")

# ==================== 2. تعريف الشخصيات بالعربية ====================
CHARACTERS_AR = [
    # 🛡️ المديرين (الحماة الاستباقيين)
    "الناقد الداخلي", "الكمالي", "المُرضي للناس", "المتحكّم",
    "الجزء البارد", "مدمن الشغل", "الجزء الحيران",
    # 🔥 رجال الإطفاء (الحماة التفاعليين)
    "المؤجِّل", "الآكل/النهم", "اللاعب المفرط",
    # 😥 المنفيين (الأجزاء المجروحة)
    "الجزء الوحيد", "الجزء الخايف", "الجزء المهمَل", "الجزء الخجول",
    "الجزء المرهَق", "الجزء المعتمد", "الجزء الغيور", "الطفل الجريح"
]

MANAGERS_AR = ["الناقد الداخلي", "الكمالي", "المُرضي للناس", "المتحكّم",
               "الجزء البارد", "مدمن الشغل", "الجزء الحيران"]

FIREFIGHTERS_AR = ["المؤجِّل", "الآكل/النهم", "اللاعب المفرط"]

EXILES_AR = ["الجزء الوحيد", "الجزء الخايف", "الجزء المهمَل", "الجزء الخجول",
             "الجزء المرهَق", "الجزء المعتمد", "الجزء الغيور", "الطفل الجريح"]

# ==================== 3. توقيعات الشخصيات المتقدمة (العربية) ====================
CHARACTER_SIGNATURES_AR = {
    "الكمالي": {
        "must_have": ["Q1:0", "Q2:high", "Q3:0"],
        "often_have": ["Q5:0", "Q7:0", "Q10:1", "Q13:2", "Q12:0"],
        "contradicts": ["المؤجِّل", "اللاعب المفرط", "الآكل/النهم"],
        "confidence_range": (0.80, 0.95),
        "evidence_weight": 1.4
    },
    "الناقد الداخلي": {
        "must_have": ["Q3:3", "Q11:0"],
        "often_have": ["Q2:high", "Q5:0", "Q7:5", "Q13:5", "Q6:5"],
        "contradicts": ["المُرضي للناس", "الجزء المعتمد"],
        "confidence_range": (0.75, 0.90),
        "evidence_weight": 1.3
    },
    "المُرضي للناس": {
        "must_have": ["Q1:2", "Q10:0"],
        "often_have": ["Q7:3", "Q9:0", "Q13:3", "Q5:2", "Q11:4"],
        "contradicts": ["المتحكّم", "الجزء البارد"],
        "confidence_range": (0.75, 0.90),
        "evidence_weight": 1.3
    },
    "المتحكّم": {
        "must_have": ["Q1:0", "Q3:0"],
        "often_have": ["Q7:0", "Q10:1", "Q13:2", "Q12:0"],
        "contradicts": ["الجزء المعتمد", "المؤجِّل"],
        "confidence_range": (0.70, 0.85),
        "evidence_weight": 1.2
    },
    "المؤجِّل": {
        "must_have": ["Q1:3", "Q8:high"],
        "often_have": ["Q3:1", "Q7:4", "Q13:4", "Q5:3"],
        "contradicts": ["الكمالي", "مدمن الشغل", "المتحكّم"],
        "confidence_range": (0.70, 0.85),
        "evidence_weight": 1.2
    },
    "الجزء الوحيد": {
        "must_have": ["Q4:high", "Q11:1"],
        "often_have": ["Q1:5", "Q12:3", "Q13:0", "Q9:4", "Q6:3"],
        "contradicts": ["الجزء البارد", "مدمن الشغل"],
        "confidence_range": (0.80, 0.95),
        "evidence_weight": 1.4
    },
    "الجزء الخايف": {
        "must_have": ["Q6:1", "Q11:2"],
        "often_have": ["Q9:1", "Q13:0", "Q12:2", "Q3:2"],
        "contradicts": ["المتحكّم", "الجزء البارد"],
        "confidence_range": (0.70, 0.85),
        "evidence_weight": 1.1
    },
    "الجزء المرهَق": {
        "must_have": ["Q6:4", "Q13:0"],
        "often_have": ["Q11:2", "Q12:4", "Q5:4", "Q9:5"],
        "contradicts": ["المتحكّم", "الجزء البارد"],
        "confidence_range": (0.65, 0.80),
        "evidence_weight": 1.1
    },
    "الجزء المعتمد": {
        "must_have": ["Q9:5", "Q12:4"],
        "often_have": ["Q6:1", "Q10:0", "Q13:3", "Q7:3"],
        "contradicts": ["المتحكّم", "الجزء البارد", "الناقد الداخلي"],
        "confidence_range": (0.65, 0.80),
        "evidence_weight": 1.0
    },
    "الجزء الخجول": {
        "must_have": ["Q11:0", "Q6:5"],
        "often_have": ["Q9:2", "Q10:5", "Q13:1", "Q7:5"],
        "contradicts": ["المُرضي للناس", "مدمن الشغل"],
        "confidence_range": (0.70, 0.85),
        "evidence_weight": 1.1
    },
    "الجزء المهمَل": {
        "must_have": ["Q3:4", "Q11:3"],
        "often_have": ["Q6:3", "Q4:high", "Q13:7", "Q1:5"],
        "contradicts": ["مدمن الشغل", "المُرضي للناس"],
        "confidence_range": (0.70, 0.85),
        "evidence_weight": 1.1
    },
    "الجزء الحيران": {
        "must_have": ["Q1:5", "Q5:5"],
        "often_have": ["Q3:4", "Q11:5", "Q13:4", "Q12:5"],
        "contradicts": ["المتحكّم", "الكمالي"],
        "confidence_range": (0.50, 0.70),
        "evidence_weight": 0.9
    },
    "الجزء البارد": {
        "must_have": ["Q1:4", "Q7:2"],
        "often_have": ["Q3:3", "Q13:7", "Q10:1"],
        "contradicts": ["الجزء المعتمد", "الجزء المرهَق", "المُرضي للناس"],
        "confidence_range": (0.65, 0.80),
        "evidence_weight": 1.0
    },
    "مدمن الشغل": {
        "must_have": ["Q1:1", "Q7:0"],
        "often_have": ["Q12:0", "Q2:high", "Q10:1"],
        "contradicts": ["المؤجِّل", "اللاعب المفرط", "الآكل/النهم"],
        "confidence_range": (0.70, 0.85),
        "evidence_weight": 1.2
    },
    "الآكل/النهم": {
        "must_have": ["Q7:1", "Q8:high"],
        "often_have": ["Q3:1", "Q13:1", "Q11:4"],
        "contradicts": ["الكمالي", "مدمن الشغل"],
        "confidence_range": (0.65, 0.80),
        "evidence_weight": 1.0
    },
    "اللاعب المفرط": {
        "must_have": ["Q7:4", "Q8:high"],
        "often_have": ["Q1:3", "Q13:4", "Q3:1"],
        "contradicts": ["الكمالي", "مدمن الشغل", "المتحكّم"],
        "confidence_range": (0.65, 0.80),
        "evidence_weight": 1.0
    },
    "الطفل الجريح": {
        "must_have": ["Q3:2", "Q6:5"],
        "often_have": ["Q9:4", "Q12:1", "Q11:0"],
        "contradicts": ["المتحكّم", "الجزء البارد"],
        "confidence_range": (0.70, 0.85),
        "evidence_weight": 1.1
    },
    "الجزء الغيور": {
        "must_have": ["Q10:2", "Q13:1"],
        "often_have": ["Q9:2", "Q10:3", "Q11:0"],
        "contradicts": ["المُرضي للناس", "الجزء المعتمد"],
        "confidence_range": (0.60, 0.75),
        "evidence_weight": 0.8
    }
}

# ==================== 4. علاقات الشخصيات (العربية) ====================
CHARACTER_RELATIONSHIPS_AR = {
    "الكمالي": {
        "strong_relations": ["المتحكّم", "الناقد الداخلي", "مدمن الشغل"],
        "common_companions": ["الجزء الخجول", "الجزء الخايف"],
        "rare_with": ["المؤجِّل", "اللاعب المفرط", "الآكل/النهم"]
    },
    "الناقد الداخلي": {
        "strong_relations": ["الجزء الخجول", "الكمالي", "الجزء الخايف"],
        "common_companions": ["الطفل الجريح", "المتحكّم"],
        "rare_with": ["المُرضي للناس", "الجزء المعتمد"]
    },
    "المُرضي للناس": {
        "strong_relations": ["الجزء المعتمد", "الجزء الخايف", "الجزء المرهَق"],
        "common_companions": ["الناقد الداخلي", "الجزء الخجول"],
        "rare_with": ["المتحكّم", "الجزء البارد", "مدمن الشغل"]
    },
    "المتحكّم": {
        "strong_relations": ["الكمالي", "الناقد الداخلي", "الجزء البارد"],
        "common_companions": ["مدمن الشغل", "الجزء الخايف"],
        "rare_with": ["الجزء المعتمد", "المؤجِّل", "الجزء الحيران"]
    },
    "المؤجِّل": {
        "strong_relations": ["اللاعب المفرط", "الآكل/النهم", "الجزء الخايف"],
        "common_companions": ["الجزء الحيران", "الجزء المرهَق"],
        "rare_with": ["الكمالي", "مدمن الشغل", "المتحكّم"]
    },
    "الجزء الوحيد": {
        "strong_relations": ["الجزء المهمَل", "الطفل الجريح", "الجزء الخايف"],
        "common_companions": ["الجزء المرهَق", "الجزء المعتمد"],
        "rare_with": ["الجزء البارد", "مدمن الشغل", "المتحكّم"]
    },
    "الجزء الخايف": {
        "strong_relations": ["الجزء المرهَق", "الجزء المعتمد", "الجزء الوحيد"],
        "common_companions": ["الناقد الداخلي", "الجزء الخجول"],
        "rare_with": ["المتحكّم", "الجزء البارد", "مدمن الشغل"]
    },
    "الجزء المرهَق": {
        "strong_relations": ["الجزء الخايف", "الجزء الحيران", "الجزء المهمَل"],
        "common_companions": ["الجزء المعتمد", "الجزء الوحيد"],
        "rare_with": ["المتحكّم", "الجزء البارد", "مدمن الشغل"]
    },
    "الجزء المعتمد": {
        "strong_relations": ["المُرضي للناس", "الجزء الخايف", "الجزء المرهَق"],
        "common_companions": ["الجزء الوحيد", "الجزء المهمَل"],
        "rare_with": ["المتحكّم", "الجزء البارد", "الناقد الداخلي"]
    }
}

# ==================== 5. هندسة الميزات المتقدمة (العربية) ====================
print("\n2. إنشاء الميزات المتقدمة...")

class AdvancedFeatureEngineerAr:
    """يُنشئ ميزات ذات معنى نفسي بالعربية"""

    def __init__(self):
        self.slider_map = {'0-20%': 0.1, '21-50%': 0.35, '51-80%': 0.65, '81-100%': 0.9}
        self.characters = CHARACTERS_AR

    def create_features(self, df):
        """إنشاء جميع الميزات من الإجابات الأولية"""
        features = pd.DataFrame(index=df.index)

        # 1. التحويلات العددية الأساسية
        for q in ['Q2', 'Q4', 'Q8']:
            if q in df.columns:
                features[f'{q}_num'] = df[q].map(self.slider_map).fillna(0.5)

        # 2. ميزات العد مع معنى نفسي
        for q in ['Q1', 'Q3', 'Q5', 'Q6', 'Q7', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13']:
            if q in df.columns:
                features[f'{q}_count'] = df[q].apply(lambda x: len(str(x).split(',')) if pd.notna(x) else 0)

        # 3. درجات الأبعاد النفسية
        # البدء بقيم صفرية أولاً
        features['perfectionism_score'] = 0
        features['loneliness_score'] = 0
        features['escapism_score'] = 0
        features['self_criticism_score'] = 0
        features['social_focus_score'] = 0
        features['control_score'] = 0
        features['vulnerability_score'] = 0

        # 4. مؤشرات الخيارات الرئيسية
        key_options = {
            'Q1': ['0', '2', '3', '5'],
            'Q3': ['0', '1', '2', '3', '4'],
            'Q5': ['0', '1', '2', '3', '4', '5'],
            'Q7': ['0', '1', '3', '4', '5'],
            'Q10': ['0', '1', '2', '3', '4'],
            'Q11': ['0', '1', '2', '3', '4', '5'],
            'Q12': ['0', '1', '2', '3', '4', '5'],
            'Q13': ['0', '1', '2', '4', '5', '7']
        }

        for q, options in key_options.items():
            if q in df.columns:
                for option in options:
                    col_name = f'{q}_opt_{option}'
                    features[col_name] = df[q].apply(
                        lambda x: 1 if option in str(x).split(',') else 0
                    )

        # 5. مؤشرات وضوح الأنماط
        features['clear_perfectionist'] = (
            (features['Q2_num'] > 0.8) &
            (features.get('Q1_opt_0', 0) == 1) &
            (features.get('Q3_opt_0', 0) == 1)
        ).astype(int)

        features['clear_people_pleaser'] = (
            (features.get('Q1_opt_2', 0) == 1) &
            (features.get('Q10_opt_0', 0) == 1) &
            (features.get('Q7_opt_3', 0) == 1)
        ).astype(int)

        features['clear_procrastinator'] = (
            (features['Q8_num'] > 0.8) &
            (features.get('Q1_opt_3', 0) == 1) &
            (features.get('Q7_opt_4', 0) == 1)
        ).astype(int)

        features['clear_lonely'] = (
            (features['Q4_num'] > 0.8) &
            (features.get('Q11_opt_1', 0) == 1) &
            (features.get('Q12_opt_3', 0) == 1)
        ).astype(int)

        features['clear_inner_critic'] = (
            (features.get('Q3_opt_3', 0) == 1) &
            (features.get('Q11_opt_0', 0) == 1) &
            (features.get('Q7_opt_5', 0) == 1)
        ).astype(int)

        # 6. حساب الدرجات النفسية بعد إنشاء مؤشرات الخيارات
        features['perfectionism_score'] = (
            features['Q2_num'].fillna(0) * 0.4 +
            features.get('Q1_opt_0', 0) * 0.3 +
            features.get('Q3_opt_0', 0) * 0.3
        )

        features['loneliness_score'] = (
            features['Q4_num'].fillna(0) * 0.5 +
            features.get('Q11_opt_1', 0) * 0.3 +
            features.get('Q12_opt_3', 0) * 0.2
        )

        features['escapism_score'] = (
            features['Q8_num'].fillna(0) * 0.5 +
            features.get('Q1_opt_3', 0) * 0.2 +
            features.get('Q7_opt_4', 0) * 0.2 +
            features.get('Q7_opt_1', 0) * 0.1
        )

        features['self_criticism_score'] = (
            features.get('Q11_opt_0', 0) * 0.4 +
            features.get('Q7_opt_5', 0) * 0.3 +
            features.get('Q3_opt_3', 0) * 0.3
        )

        features['social_focus_score'] = (
            features.get('Q1_opt_2', 0) * 0.4 +
            features.get('Q10_opt_0', 0) * 0.3 +
            features.get('Q7_opt_3', 0) * 0.3
        )

        features['control_score'] = (
            features.get('Q1_opt_0', 0) * 0.4 +
            features.get('Q7_opt_0', 0) * 0.3 +
            features.get('Q10_opt_1', 0) * 0.3
        )

        features['vulnerability_score'] = (
            features.get('Q3_opt_2', 0) * 0.3 +
            features.get('Q6_opt_1', 0) * 0.3 +
            features.get('Q9_opt_4', 0) * 0.2 +
            features.get('Q13_opt_0', 0) * 0.2
        )

        # 6. مقاييس الغموض والوضوح
        clear_pattern_cols = [c for c in features.columns if c.startswith('clear_')]
        if clear_pattern_cols:
            features['clear_pattern_count'] = features[clear_pattern_cols].sum(axis=1)
            features['has_clear_pattern'] = (features['clear_pattern_count'] > 0).astype(int)
            features['has_multiple_clear'] = (features['clear_pattern_count'] > 1).astype(int)

        # 7. مقاييس تناسق الاستجابات
        slider_cols = [c for c in features.columns if c.endswith('_num')]
        if len(slider_cols) > 1:
            features['slider_consistency'] = 1 - features[slider_cols].std(axis=1).fillna(0)

        count_cols = [c for c in features.columns if c.endswith('_count')]
        if len(count_cols) > 1:
            features['selection_consistency'] = 1 - (features[count_cols].std(axis=1) / 3).fillna(0)

        # 8. هيمنة الأنماط
        manager_indicators = features.get('clear_perfectionist', 0) + \
                           features.get('clear_people_pleaser', 0) + \
                           features.get('clear_inner_critic', 0) + \
                           features.get('Q1_opt_0', 0)

        firefighter_indicators = features.get('clear_procrastinator', 0) + \
                                (features['Q8_num'] > 0.7).astype(int)

        exile_indicators = features.get('clear_lonely', 0) + \
                          (features['Q4_num'] > 0.7).astype(int)

        total_indicators = manager_indicators + firefighter_indicators + exile_indicators
        features['archetype_clarity'] = np.where(
            total_indicators > 0,
            np.maximum(manager_indicators, np.maximum(firefighter_indicators, exile_indicators)) / total_indicators,
            0.5
        )

        # 9. درجة الغموض الكلي (0 = واضح جداً، 1 = غامض جداً)
        features['total_ambiguity'] = (
            (features.get('clear_pattern_count', 0) == 0).astype(float) * 0.3 +
            features.get('has_multiple_clear', 0).astype(float) * 0.3 +
            (features.get('slider_consistency', 0.5) < 0.7).astype(float) * 0.2 +
            (features.get('selection_consistency', 0.5) < 0.6).astype(float) * 0.2
        )

        # 10. مؤشرات التوتر النفسي
        features['perfection_vs_procrastination'] = (
            features['perfectionism_score'] * features['escapism_score']
        )

        features['control_vs_vulnerability'] = (
            features['control_score'] * features['vulnerability_score']
        )

        features['inner_conflict_score'] = (
            features['perfection_vs_procrastination'] * 0.4 +
            features['control_vs_vulnerability'] * 0.3 +
            features['total_ambiguity'] * 0.3
        )

        # تعبئة القيم NaN
        features = features.fillna(0)

        # قص القيم إلى نطاقات معقولة
        for col in features.columns:
            if features[col].dtype in ['float64', 'float32']:
                features[col] = np.clip(features[col], 0, 1)

        return features

# ==================== 6. مصنف أنواع الأنماط (العربية) ====================
class PatternTypeClassifierAr:
    """يصنف أنماط الإجابات إلى أنواع الأنماط النفسية بالعربية"""

    def __init__(self):
        self.pattern_types = ['very_clear', 'clear', 'mixed', 'ambiguous', 'very_mixed', 'contradictory']

    def classify(self, features: pd.Series) -> str:
        """تصنيف نوع النمط بناءً على الميزات"""

        # استخراج المقاييس الرئيسية
        clear_count = features.get('clear_pattern_count', 0)
        ambiguity = features.get('total_ambiguity', 0.5)
        conflict = features.get('inner_conflict_score', 0)
        clarity = features.get('archetype_clarity', 0.5)

        # التحقق من الأنماط المتناقضة أولاً
        if conflict > 0.7:
            return 'contradictory'

        # واضح جداً: نمط واحد مهيمن مع غموض منخفض
        if clear_count == 1 and ambiguity < 0.3 and clarity > 0.8:
            return 'very_clear'

        # واضح: نمط واضح لكن مع بعض التعقيد
        if clear_count == 1 and ambiguity < 0.4 and clarity > 0.6:
            return 'clear'

        # مختلط: أنماط واضحة متعددة
        if clear_count >= 2 and ambiguity < 0.6:
            return 'mixed'

        # مختلط جداً: أنماط كثيرة مع غموض عالٍ
        if clear_count >= 3 or ambiguity > 0.8:
            return 'very_mixed'

        # غامض: غير واضح لكن ليس مختلطاً بشدة
        if clear_count == 0 and ambiguity > 0.4:
            return 'ambiguous'

        # الافتراضي مختلط
        return 'mixed'

# ==================== 7. مدقق الإجابات (العربية) ====================
class AnswerValidatorAr:
    """يُدقق إجابات المستخدم للتناسق النفسي بالعربية"""

    def __init__(self):
        self.slider_map = {'0-20%': 0.1, '21-50%': 0.35, '51-80%': 0.65, '81-100%': 0.9}

    def validate(self, user_answers: Dict) -> List[str]:
        """تدقيق إجابات المستخدم، إرجاع قائمة بالتحذيرات/المشكلات"""
        warnings = []

        # 1. التحقق من كثرة الاختيارات (ربما تشير إلى ضوضاء)
        total_selections = 0
        for q, val in user_answers.items():
            if isinstance(val, str):
                selections = len(val.split(','))
                total_selections += selections

                # فحوصات السؤال الفردي
                if selections > 3 and q in ['Q1', 'Q3', 'Q5', 'Q7']:
                    warnings.append(f"السؤال {q}: اختيارات كثيرة ({selections})، قد تشير إلى التردد")

        if total_selections > 30:
            warnings.append(f"اختيارات إجمالية كثيرة جداً ({total_selections})، قد تكون الإجابات غير دقيقة")

        # 2. التحقق من تناسق أشرطة التمرير
        slider_values = {}
        for q in ['Q2', 'Q4', 'Q8']:
            if q in user_answers:
                slider_values[q] = self.slider_map.get(user_answers[q], 0.5)

        if len(slider_values) == 3:
            # التحقق من عدم التناسق الشديد
            values = list(slider_values.values())
            if max(values) - min(values) > 0.8:
                warnings.append("عدم تناسق شديد في استجابات أشرطة التمرير")

        # 3. التحقق من الأنماط المتناقضة
        contradictory_pairs = self._check_contradictions(user_answers)
        if contradictory_pairs:
            warnings.append(f"تم اكتشاف أنماط متناقضة: {', '.join(contradictory_pairs)}")

        # 4. التحقق من أنماط "اختيار الكل"
        all_selected_questions = []
        for q in ['Q1', 'Q3', 'Q5', 'Q6', 'Q7', 'Q9', 'Q10', 'Q11', 'Q12']:
            if q in user_answers:
                if len(user_answers[q].split(',')) >= 5:  # معظم أو كل الخيارات محددة
                    all_selected_questions.append(q)

        if len(all_selected_questions) >= 3:
            warnings.append(f"أسئلة متعددة مع تحديد معظم الخيارات: {', '.join(all_selected_questions)}")

        return warnings

    def _check_contradictions(self, user_answers: Dict) -> List[str]:
        """التحقق من الإجابات النفسية المتناقضة"""
        contradictions = []

        # تناقض الكمالي + المؤجل
        if self._has_perfectionist_signature(user_answers) and self._has_procrastinator_signature(user_answers):
            contradictions.append("الكمالي-المؤجل")

        # تناقض المتحكم + المعتمد
        if self._has_controller_signature(user_answers) and self._has_dependent_signature(user_answers):
            contradictions.append("المتحكم-المعتمد")

        # تناقض البارد + المرهق
        if self._has_stoic_signature(user_answers) and self._has_overwhelmed_signature(user_answers):
            contradictions.append("البارد-المرهق")

        return contradictions

    def _has_perfectionist_signature(self, answers):
        q2_val = self.slider_map.get(answers.get('Q2', '21-50%'), 0.35)
        q1_has_0 = '0' in str(answers.get('Q1', '')).split(',')
        return q2_val > 0.8 and q1_has_0

    def _has_procrastinator_signature(self, answers):
        q8_val = self.slider_map.get(answers.get('Q8', '21-50%'), 0.35)
        q1_has_3 = '3' in str(answers.get('Q1', '')).split(',')
        return q8_val > 0.8 and q1_has_3

    def _has_controller_signature(self, answers):
        q1_has_0 = '0' in str(answers.get('Q1', '')).split(',')
        q3_has_0 = '0' in str(answers.get('Q3', '')).split(',')
        return q1_has_0 and q3_has_0

    def _has_dependent_signature(self, answers):
        q9_has_5 = '5' in str(answers.get('Q9', '')).split(',')
        q12_has_4 = '4' in str(answers.get('Q12', '')).split(',')
        return q9_has_5 or q12_has_4

    def _has_stoic_signature(self, answers):
        q1_has_4 = '4' in str(answers.get('Q1', '')).split(',')
        q7_has_2 = '2' in str(answers.get('Q7', '')).split(',')
        return q1_has_4 and q7_has_2

    def _has_overwhelmed_signature(self, answers):
        q6_has_4 = '4' in str(answers.get('Q6', '')).split(',')
        q13_has_0 = '0' in str(answers.get('Q13', '')).split(',')
        return q6_has_4 or q13_has_0

# ==================== 8. تدريب النموذج الثابت مع التعامل الصحيح للفئات (العربية) ====================
print("\n3. تدريب النموذج الإنتاجي الثابت مع التعامل الصحيح للفئات...")

class FixedProductionPredictorAr:
    """النموذج الثابت مع التعامل الصحيح للفئات لكل الشخصيات بالعربية"""

    def __init__(self, train_on_data=True):
        self.characters = CHARACTERS_AR
        self.feature_engineer = AdvancedFeatureEngineerAr()
        self.pattern_classifier = PatternTypeClassifierAr()
        self.answer_validator = AnswerValidatorAr()

        # إنشاء تعيين الشخصية إلى الفهرس
        self.char_to_idx = {char: idx for idx, char in enumerate(self.characters)}
        self.idx_to_char = {idx: char for idx, char in enumerate(self.characters)}

        if train_on_data:
            self._train_models()
        else:
            self.models = None
            self.scalers = None

    def _train_models(self):
        """تدريب النماذج مع التعامل الصحيح للفئات"""
        print("   إنشاء الميزات...")

        # إنشاء الميزات
        features = self.feature_engineer.create_features(df)

        # إنشاء الهدف مع جميع الفئات الـ 18
        y = df['top1_char'].map(self.char_to_idx).fillna(0).astype(int)

        # التأكد من وجود جميع الفئات الـ 18
        unique_classes = np.unique(y)
        print(f"   تم العثور على {len(unique_classes)} فئة فريدة في البيانات")

        # تصنيف الأنماط
        print("   تصنيف أنواع الأنماط...")
        pattern_types = features.apply(self.pattern_classifier.classify, axis=1)
        features['pattern_type'] = pattern_types

        print("   توزيع أنواع الأنماط:")
        print(pattern_types.value_counts())

        # تدريب نموذج رئيسي (باستخدام جميع البيانات)
        print("   تدريب النموذج الرئيسي...")
        X_main = features.drop('pattern_type', axis=1, errors='ignore')

        # التأكد من أعمدة الميزات المتسقة
        self.feature_columns = list(X_main.columns)

        # تقسيم التدريب والاختبار
        X_train, X_test, y_train, y_test = train_test_split(
            X_main, y, test_size=0.2, random_state=42, stratify=y
        )

        # قياس الميزات
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # تدريب نموذج التجميع
        print("   تدريب نموذج التجميع...")

        # استخدام RandomForest الذي يتعامل بشكل أفضل مع الفئات المفقودة
        rf_model = RandomForestClassifier(
            n_estimators=200,
            max_depth=15,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features='sqrt',
            random_state=42,
            n_jobs=-1,
            class_weight='balanced'
        )

        # استخدام GradientBoosting كاحتياطي
        gb_model = GradientBoostingClassifier(
            n_estimators=150,
            max_depth=6,
            learning_rate=0.05,
            subsample=0.8,
            random_state=42
        )

        # التجميع
        ensemble_model = VotingClassifier(
            estimators=[('rf', rf_model), ('gb', gb_model)],
            voting='soft',
            weights=[0.7, 0.3]
        )

        ensemble_model.fit(X_train_scaled, y_train)

        # التقييم
        y_pred = ensemble_model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"   دقة النموذج الرئيسي: {accuracy:.2%}")

        # تخزين النماذج
        self.model = ensemble_model
        self.scaler = scaler
        self.pattern_distribution = pattern_types.value_counts().to_dict()

        # تدريب نماذج الاحتياط المحددة بالأنماط فقط للأنماط الشائعة
        print("   تدريب نماذج الاحتياط المحددة بالأنماط...")
        self.pattern_models = {}
        self.pattern_scalers = {}

        # التدريب فقط للأنماط ذات العينات الكافية
        for pattern_type, count in pattern_types.value_counts().items():
            if count > 1000:  # تحتاج إلى عينات كافية
                print(f"     تدريب نموذج {pattern_type} ({count:,} عينة)...")

                # الحصول على المؤشرات لهذا النمط
                pattern_indices = pattern_types[pattern_types == pattern_type].index

                if len(pattern_indices) > 100:
                    X_pattern = X_main.loc[pattern_indices]
                    y_pattern = y.loc[pattern_indices]

                    # التحقق مما إذا كان لدينا جميع الفئات
                    unique_in_pattern = np.unique(y_pattern)
                    if len(unique_in_pattern) > 5:  # تحتاج إلى فئات متعددة
                        # تدريب نموذج بسيط
                        pattern_scaler = StandardScaler()
                        X_pattern_scaled = pattern_scaler.fit_transform(X_pattern)

                        pattern_model = RandomForestClassifier(
                            n_estimators=100,
                            max_depth=10,
                            random_state=42,
                            n_jobs=-1,
                            class_weight='balanced'
                        )

                        pattern_model.fit(X_pattern_scaled, y_pattern)

                        self.pattern_models[pattern_type] = pattern_model
                        self.pattern_scalers[pattern_type] = pattern_scaler

                        # اختبار سريع
                        if len(pattern_indices) > 200:
                            test_size = min(100, len(pattern_indices) // 5)
                            test_acc = pattern_model.score(
                                pattern_scaler.transform(X_pattern.iloc[:test_size]),
                                y_pattern.iloc[:test_size]
                            )
                            print(f"       دقة الاختبار: {test_acc:.2%}")

        print("   ✅ تم تدريب جميع النماذج بنجاح")

    def predict_with_confidence(self, user_answers: Dict) -> Dict:
        """التنبؤ الرئيسي مع معايرة الثقة"""

        # الخطوة 1: تدقيق الإجابات
        validation_warnings = self.answer_validator.validate(user_answers)

        # الخطوة 2: إنشاء الميزات
        df_input = pd.DataFrame([user_answers])
        features = self.feature_engineer.create_features(df_input)

        # التأكد من وجود جميع أعمدة الميزات
        for col in self.feature_columns:
            if col not in features.columns:
                features[col] = 0

        features = features[self.feature_columns]

        # الخطوة 3: تصنيف نوع النمط
        pattern_type = self.pattern_classifier.classify(features.iloc[0])

        # الخطوة 4: الحصول على التنبؤات من النموذج الرئيسي
        X_scaled = self.scaler.transform(features)

        # الحصول على الاحتمالات من النموذج الرئيسي
        if hasattr(self.model, 'predict_proba'):
            probabilities = self.model.predict_proba(X_scaled)[0]
        else:
            # الاحتياط: استخدام دالة القرار أو التنبؤ البسيط
            probabilities = np.zeros(len(self.characters))
            prediction = self.model.predict(X_scaled)[0]
            probabilities[prediction] = 0.8
            # إضافة بعض الاحتمالية للشخصيات المماثلة
            for i in range(len(probabilities)):
                if i != prediction:
                    probabilities[i] = 0.2 / (len(probabilities) - 1)

        # الخطوة 5: ضبط الثقة بناءً على نوع النمط
        base_confidence = np.max(probabilities)
        top_idx = np.argmax(probabilities)
        top_character = self.idx_to_char[top_idx]

        # الحصول على أفضل 3 شخصيات
        top_3_indices = np.argsort(probabilities)[-3:][::-1]
        top_3_characters = [self.idx_to_char[idx] for idx in top_3_indices]

        # الخطوة 6: معايرة الثقة بناءً على نوع النمط
        calibrated_confidence = self._calibrate_confidence(
            base_confidence, pattern_type, features.iloc[0]
        )

        # الخطوة 7: توليد تحليل الأدلة
        evidence_analysis = self._analyze_evidence(top_character, user_answers)

        # الخطوة 8: تجميع النتيجة
        result = {
            'prediction': {
                'top_character': top_character,
                'top_3_characters': top_3_characters,
                'confidence': round(calibrated_confidence, 3),
                'confidence_percent': f"{calibrated_confidence:.1%}",
                'confidence_label': self._get_confidence_label(calibrated_confidence),
                'pattern_type': pattern_type,
                'certainty_level': self._get_certainty_level(calibrated_confidence, pattern_type)
            },
            'analysis': {
                'evidence_strength': evidence_analysis['strength'],
                'evidence_score': evidence_analysis['score'],
                'pattern_clarity': features.iloc[0].get('archetype_clarity', 0.5),
                'ambiguity_score': features.iloc[0].get('total_ambiguity', 0.5)
            },
            'warnings': {
                'validation_warnings': validation_warnings,
                'has_warnings': len(validation_warnings) > 0
            },
            'raw_data': {
                'all_probabilities': {self.idx_to_char[i]: float(prob)
                                     for i, prob in enumerate(probabilities)},
                'pattern_type': pattern_type
            }
        }

        return result

    def _calibrate_confidence(self, base_confidence: float, pattern_type: str,
                            features: pd.Series) -> float:
        """معايرة الثقة بناءً على نوع النمط"""

        clarity = features.get('archetype_clarity', 0.5)
        ambiguity = features.get('total_ambiguity', 0.5)

        # التعديلات الأساسية حسب نوع النمط
        if pattern_type == 'very_clear':
            target_min, target_max = 0.85, 0.95
            if clarity > 0.9:
                base = 0.9
            elif clarity > 0.8:
                base = 0.85
            else:
                base = 0.8

        elif pattern_type == 'clear':
            target_min, target_max = 0.75, 0.88
            if clarity > 0.8:
                base = 0.82
            elif clarity > 0.7:
                base = 0.78
            else:
                base = 0.75

        elif pattern_type == 'mixed':
            target_min, target_max = 0.50, 0.70
            clear_count = features.get('clear_pattern_count', 0)
            if clear_count == 2 and ambiguity < 0.5:
                base = 0.65
            elif clear_count >= 3:
                base = 0.55
            else:
                base = 0.60

        elif pattern_type == 'ambiguous':
            target_min, target_max = 0.30, 0.50
            if clarity < 0.4:
                base = 0.35
            elif ambiguity > 0.7:
                base = 0.3
            else:
                base = 0.4

        else:  # very_mixed or contradictory
            target_min, target_max = 0.15, 0.35
            clear_count = features.get('clear_pattern_count', 0)
            if clear_count >= 3 or ambiguity > 0.8:
                base = 0.2
            else:
                base = 0.25

        # المزج مع ثقة النموذج
        calibrated = (base * 0.6) + (base_confidence * 0.4)

        # إضافة تباين عشوائي صغير للواقعية
        if pattern_type in ['mixed', 'ambiguous', 'very_mixed']:
            calibrated += np.random.uniform(-0.05, 0.05)

        # التأكد من البقاء ضمن الحدود
        calibrated = np.clip(calibrated, target_min, target_max)

        return calibrated

    def _analyze_evidence(self, character: str, user_answers: Dict) -> Dict:
        """تحليل الأدلة لشخصية"""

        if character not in CHARACTER_SIGNATURES_AR:
            return {'score': 0, 'strength': 'very_weak'}

        sig_info = CHARACTER_SIGNATURES_AR[character]
        must_matched = 0
        often_matched = 0

        slider_map = {'0-20%': 0.1, '21-50%': 0.35, '51-80%': 0.65, '81-100%': 0.9}

        # التحقق من الشروط الضرورية
        for condition in sig_info.get("must_have", []):
            q, val = condition.split(':')
            if q in user_answers:
                if val == 'high':
                    if q in ['Q2', 'Q4', 'Q8']:
                        num_val = slider_map.get(user_answers[q], 0.5)
                        if num_val > 0.8:
                            must_matched += 1
                elif val == 'low':
                    if q in ['Q2', 'Q4', 'Q8']:
                        num_val = slider_map.get(user_answers[q], 0.5)
                        if num_val < 0.2:
                            must_matched += 1
                else:
                    if val in str(user_answers[q]).split(','):
                        must_matched += 1

        # التحقق من الشروط المتكررة
        for condition in sig_info.get("often_have", []):
            q, val = condition.split(':')
            if q in user_answers:
                if val == 'high':
                    if q in ['Q2', 'Q4', 'Q8']:
                        num_val = slider_map.get(user_answers[q], 0.5)
                        if num_val > 0.8:
                            often_matched += 1
                elif val == 'low':
                    if q in ['Q2', 'Q4', 'Q8']:
                        num_val = slider_map.get(user_answers[q], 0.5)
                        if num_val < 0.2:
                            often_matched += 1
                else:
                    if val in str(user_answers[q]).split(','):
                        often_matched += 1

        # حساب النتيجة
        total_must = len(sig_info.get("must_have", []))
        total_often = len(sig_info.get("often_have", []))

        if total_must > 0:
            must_score = must_matched / total_must
            often_score = often_matched / total_often if total_often > 0 else 0
            score = (must_score * 0.7) + (often_score * 0.3)
        else:
            score = often_matched / total_often if total_often > 0 else 0

        # تحديد القوة
        if must_matched >= 2 or score > 0.9:
            strength = 'very_strong'
        elif must_matched >= 1 or score > 0.75:
            strength = 'strong'
        elif score > 0.6:
            strength = 'moderate'
        elif score > 0.4:
            strength = 'weak'
        else:
            strength = 'very_weak'

        return {
            'score': min(score, 1.0),
            'strength': strength,
            'must_matched': must_matched,
            'often_matched': often_matched
        }

    def _get_confidence_label(self, confidence: float) -> str:
        """تحويل الثقة إلى وصف قابل للقراءة بالعربية"""
        if confidence >= 0.9:
            return "ثقة عالية جداً"
        elif confidence >= 0.85:
            return "ثقة عالية"
        elif confidence >= 0.8:
            return "ثقة متوسطة-عالية"
        elif confidence >= 0.7:
            return "ثقة متوسطة"
        elif confidence >= 0.6:
            return "ثقة منخفضة-متوسطة"
        elif confidence >= 0.5:
            return "ثقة منخفضة"
        elif confidence >= 0.3:
            return "ثقة منخفضة جداً"
        else:
            return "ثقة ضئيلة"

    def _get_certainty_level(self, confidence: float, pattern_type: str) -> str:
        """الحصول على مستوى اليقين بالعربية"""
        if confidence >= 0.85:
            return "يقين عالي جداً"
        elif confidence >= 0.75:
            return "يقين عالي"
        elif confidence >= 0.65:
            return "يقين متوسط"
        elif confidence >= 0.5:
            return "يقين منخفض"
        else:
            return "يقين منخفض جداً"

    def save_model(self, path='fixed_production_predictor_ar.pkl'):
        """حفظ النموذج المدرب"""
        with open(path, 'wb') as f:
            pickle.dump({
                'model': self.model,
                'scaler': self.scaler,
                'pattern_models': self.pattern_models,
                'pattern_scalers': self.pattern_scalers,
                'feature_columns': self.feature_columns,
                'char_to_idx': self.char_to_idx,
                'idx_to_char': self.idx_to_char,
                'pattern_distribution': self.pattern_distribution
            }, f)
        print(f"   ✅ تم حفظ النموذج في {path}")

    def load_model(self, path='fixed_production_predictor_ar.pkl'):
        """تحميل نموذج مدرب"""
        with open(path, 'rb') as f:
            data = pickle.load(f)

        self.model = data['model']
        self.scaler = data['scaler']
        self.pattern_models = data.get('pattern_models', {})
        self.pattern_scalers = data.get('pattern_scalers', {})
        self.feature_columns = data['feature_columns']
        self.char_to_idx = data['char_to_idx']
        self.idx_to_char = data['idx_to_char']
        self.pattern_distribution = data.get('pattern_distribution', {})
        print(f"   ✅ تم تحميل النموذج من {path}")

# ==================== 9. تدريب النموذج الثابت بالعربية ====================
print("\n" + "="*80)
print("🚀 تدريب النموذج الإنتاجي الثابت بالعربية")
print("="*80)

# إنشاء وتدريب النموذج الثابت
predictor_ar = FixedProductionPredictorAr(train_on_data=True)

# حفظ النموذج
predictor_ar.save_model('fixed_production_character_predictor_ar.pkl')

# ==================== 10. الاختبار الشامل بالعربية ====================
print("\n" + "="*80)
print("🧪 اختبار النموذج الإنتاجي الثابت بالعربية")
print("="*80)

# حالات الاختبار بالعربية
test_cases_ar = [
    {
        "name": "اختبار 1: الكمالي الواضح جداً",
        "answers": {
            "Q1": "0", "Q2": "81-100%", "Q3": "0", "Q4": "0-20%",
            "Q5": "0", "Q6": "0", "Q7": "0", "Q8": "0-20%",
            "Q9": "3", "Q10": "1", "Q11": "0", "Q12": "0", "Q13": "2"
        }
    },
    {
        "name": "اختبار 2: المُرضي للناس الواضح",
        "answers": {
            "Q1": "2", "Q2": "51-80%", "Q3": "3", "Q4": "21-50%",
            "Q5": "2", "Q6": "2", "Q7": "3", "Q8": "21-50%",
            "Q9": "0", "Q10": "0", "Q11": "4", "Q12": "4", "Q13": "3"
        }
    },
    {
        "name": "اختبار 3: مختلط (الكمالي + المتحكّم)",
        "answers": {
            "Q1": "0", "Q2": "81-100%", "Q3": "0", "Q4": "0-20%",
            "Q5": "0,3", "Q6": "0", "Q7": "0,2", "Q8": "21-50%",
            "Q9": "3", "Q10": "1,3", "Q11": "0", "Q12": "0,2", "Q13": "2,7"
        }
    },
    {
        "name": "اختبار 4: غامض (أنماط متعددة)",
        "answers": {
            "Q1": "0,2,3", "Q2": "51-80%", "Q3": "0,1,3", "Q4": "51-80%",
            "Q5": "0,1,3", "Q6": "0,2,5", "Q7": "0,1,4", "Q8": "51-80%",
            "Q9": "0,1,4", "Q10": "0,2,4", "Q11": "0,1,3", "Q12": "0,3,5", "Q13": "0,2,4,7"
        }
    },
    {
        "name": "اختبار 5: مختلط جداً (كل الخيارات)",
        "answers": {
            "Q1": "0,1,2,3,4,5", "Q2": "51-80%", "Q3": "0,1,2,3,4,5",
            "Q4": "51-80%", "Q5": "0,1,2,3,4,5", "Q6": "0,1,2,3,4,5",
            "Q7": "0,1,2,3,4,5", "Q8": "51-80%", "Q9": "0,1,2,3,4,5",
            "Q10": "0,1,2,3,4,5", "Q11": "0,1,2,3,4,5",
            "Q12": "0,1,2,3,4,5", "Q13": "0,1,2,3,4,5,6,7"
        }
    }
]

print("\nتشغيل الاختبارات...")
print("-" * 80)

results_ar = []
for i, test in enumerate(test_cases_ar, 1):
    print(f"\n{i}. {test['name']}")
    print("-" * 40)

    try:
        result = predictor_ar.predict_with_confidence(test['answers'])
        pred = result['prediction']
        analysis = result['analysis']

        # عرض النتائج
        print(f"التنبؤ: {pred['top_character']} ({pred['confidence_percent']})")
        print(f"نوع النمط: {pred['pattern_type'].upper()}")
        print(f"تصنيف الثقة: {pred['confidence_label']}")
        print(f"مستوى اليقين: {pred['certainty_level']}")
        print(f"أفضل 3: {', '.join(pred['top_3_characters'])}")
        print(f"قوة الأدلة: {analysis['evidence_strength'].upper()}")
        print(f"نتيجة الأدلة: {analysis['evidence_score']:.2f}")

        # عرض التحذيرات إن وجدت
        if result['warnings']['has_warnings']:
            print(f"⚠️  تحذيرات:")
            for warning in result['warnings']['validation_warnings']:
                print(f"   • {warning}")

        # التحقق من الواقعية النفسية
        if pred['pattern_type'] in ['very_clear', 'clear']:
            expected_range = (0.75, 0.95)
        elif pred['pattern_type'] == 'mixed':
            expected_range = (0.50, 0.70)
        elif pred['pattern_type'] == 'ambiguous':
            expected_range = (0.30, 0.50)
        else:  # very_mixed
            expected_range = (0.15, 0.35)

        confidence = pred['confidence']
        in_range = expected_range[0] <= confidence <= expected_range[1]

        if in_range:
            print(f"✅ الثقة واقعية لنمط {pred['pattern_type']}")
        else:
            print(f"⚠️  الثقة خارج النطاق المتوقع لنمط {pred['pattern_type']}")

        # تخزين النتائج
        results_ar.append({
            'test': test['name'],
            'top_predicted': pred['top_character'],
            'confidence': confidence,
            'pattern_type': pred['pattern_type'],
            'evidence_strength': analysis['evidence_strength'],
            'confidence_realistic': in_range
        })

    except Exception as e:
        print(f"❌ خطأ: {e}")
        continue

# ==================== 11. تحليل الأداء بالعربية ====================
print("\n" + "="*80)
print("📊 تحليل الأداء")
print("="*80)

if results_ar:
    print(f"\n🎯 ملخص نتائج الاختبار:")
    print(f"   إجمالي الاختبارات: {len(test_cases_ar)}")
    print(f"   الاختبارات الناجحة: {len(results_ar)}")

    # واقعية الثقة
    realistic_count = sum(1 for r in results_ar if r['confidence_realistic'])
    realism_percentage = realistic_count / len(results_ar) * 100
    print(f"   الثقة الواقعية: {realistic_count}/{len(results_ar)} ({realism_percentage:.1f}%)")

    # توزيع قوة الأدلة
    print(f"\n🔍 توزيع قوة الأدلة:")
    strength_counts = Counter([r['evidence_strength'] for r in results_ar])
    for strength, count in strength_counts.items():
        percentage = (count / len(results_ar)) * 100
        print(f"   {strength.upper():15}: {count} اختبار ({percentage:.1f}%)")

    # توزيع أنواع الأنماط
    print(f"\n📊 توزيع أنواع الأنماط:")
    pattern_counts = Counter([r['pattern_type'] for r in results_ar])
    for pattern, count in pattern_counts.items():
        percentage = (count / len(results_ar)) * 100
        print(f"   {pattern.upper():15}: {count} اختبار ({percentage:.1f}%)")

    # الثقة حسب نوع النمط
    print(f"\n📈 متوسط الثقة حسب نوع النمط:")
    pattern_confidences = defaultdict(list)
    for result in results_ar:
        pattern_confidences[result['pattern_type']].append(result['confidence'])

    for pattern, confidences in pattern_confidences.items():
        avg_conf = np.mean(confidences) * 100
        min_conf = min(confidences) * 100
        max_conf = max(confidences) * 100
        print(f"   {pattern.upper():15}: {avg_conf:.1f}% (المدى: {min_conf:.1f}%-{max_conf:.1f}%)")

    # التقييم العام
    print(f"\n🏆 التقييم العام:")

    if realism_percentage >= 80:
        print("   ✅ ممتاز: معايرة الثقة تعمل بشكل جيد")
        print("   🚀 النموذج جاهز للإنتاج بثقة واقعية")
    elif realism_percentage >= 60:
        print("   ⚠️  جيد: تحتاج معايرة الثقة إلى تعديلات طفيفة")
        print("   📈 يظهر النموذج إمكانات بثقة واقعية إلى حد كبير")
    else:
        print("   ❌ يحتاج للتحسين: تحتاج معايرة الثقة إلى عمل")
        print("   🔧 فكر في ضبط معلمات معايرة الثقة")

    print(f"\n💡 التوصيات:")
    print("   1. مراقبة معايرة الثقة في الإنتاج")
    print("   2. جمع ملاحظات المستخدمين حول دقة التنبؤ")
    print("   3. ضبط نطاقات الثقة بناءً على الاستخدام الفعلي")
    print("   4. التفكير في اختبار A/B لمستويات الثقة المختلفة")

else:
    print("❌ لا توجد نتائج للتحليل")

# ==================== 12. عرض توضيحي بمستخدم عشوائي بالعربية ====================
print("\n" + "="*80)
print("👤 العرض التوضيحي: التنبؤ بمستخدم عشوائي")
print("="*80)

# إنشاء ملف تعريف مستخدم عشوائي ولكن واقعي نفسياً
def create_random_user_profile_ar():
    """إنشاء ملف تعريف مستخدم عشوائي ولكن واقعي نفسياً بالعربية"""

    # اختيار شخصية مهيمنة
    dominant_chars = ["الكمالي", "المُرضي للناس", "الجزء الوحيد",
                     "المؤجِّل", "الناقد الداخلي", "المتحكّم"]
    dominant = random.choice(dominant_chars)

    # إنشاء إجابات بناءً على الشخصية المهيمنة
    answers = {}

    # أشرطة التمرير
    if dominant == "الكمالي":
        answers['Q2'] = random.choice(["81-100%", "81-100%", "51-80%"])
        answers['Q4'] = random.choice(["0-20%", "21-50%"])
        answers['Q8'] = random.choice(["0-20%", "21-50%"])
    elif dominant == "المُرضي للناس":
        answers['Q2'] = random.choice(["21-50%", "51-80%"])
        answers['Q4'] = random.choice(["21-50%", "51-80%"])
        answers['Q8'] = random.choice(["21-50%", "51-80%"])
    elif dominant == "الجزء الوحيد":
        answers['Q2'] = random.choice(["21-50%", "0-20%"])
        answers['Q4'] = random.choice(["81-100%", "51-80%", "81-100%"])
        answers['Q8'] = random.choice(["21-50%", "51-80%"])
    else:
        answers['Q2'] = random.choice(["21-50%", "51-80%", "0-20%", "81-100%"])
        answers['Q4'] = random.choice(["21-50%", "51-80%", "0-20%", "81-100%"])
        answers['Q8'] = random.choice(["21-50%", "51-80%", "0-20%", "81-100%"])

    # أسئلة الاختيار الفردي/المتعدد مع إجابات مناسبة للشخصية
    if dominant == "الكمالي":
        answers['Q1'] = "0"
        answers['Q3'] = "0"
        answers['Q5'] = "0"
        answers['Q6'] = "0"
        answers['Q7'] = "0"
        answers['Q9'] = "3"
        answers['Q10'] = "1"
        answers['Q11'] = "0"
        answers['Q12'] = "0"
        answers['Q13'] = "2"
    elif dominant == "المُرضي للناس":
        answers['Q1'] = "2"
        answers['Q3'] = "3"
        answers['Q5'] = "2"
        answers['Q6'] = "2"
        answers['Q7'] = "3"
        answers['Q9'] = "0"
        answers['Q10'] = "0"
        answers['Q11'] = "4"
        answers['Q12'] = "4"
        answers['Q13'] = "3"
    elif dominant == "الجزء الوحيد":
        answers['Q1'] = "5"
        answers['Q3'] = "2,4"
        answers['Q5'] = "1"
        answers['Q6'] = "3"
        answers['Q7'] = "2"
        answers['Q9'] = "1,4"
        answers['Q10'] = "4"
        answers['Q11'] = "1"
        answers['Q12'] = "3"
        answers['Q13'] = "0,6"
    else:
        # إجابات معقولة عامة
        answers['Q1'] = random.choice(["0", "2", "3", "4", "5"])
        answers['Q3'] = random.choice(["0", "1", "2", "3", "4"])
        answers['Q5'] = random.choice(["0", "1", "2", "3", "4", "5"])
        answers['Q6'] = random.choice(["0", "1", "2", "3", "4", "5"])
        answers['Q7'] = random.choice(["0", "1", "2", "3", "4", "5"])
        answers['Q9'] = random.choice(["0", "1", "2", "3", "4", "5"])
        answers['Q10'] = random.choice(["0", "1", "2", "3", "4", "5"])
        answers['Q11'] = random.choice(["0", "1", "2", "3", "4", "5"])
        answers['Q12'] = random.choice(["0", "1", "2", "3", "4", "5"])
        answers['Q13'] = random.choice(["0", "1", "2", "3", "4", "5", "6", "7"])

    return answers, dominant

# تشغيل العرض التوضيحي
print("\nإنشاء ملف تعريف مستخدم عشوائي...")
user_answers, true_dominant = create_random_user_profile_ar()

print(f"الجزء المهيمن الحقيقي (للعرض التوضيحي): {true_dominant}")
print("\nتشغيل التنبؤ...")
print("-" * 40)

try:
    result = predictor_ar.predict_with_confidence(user_answers)
    pred = result['prediction']
    analysis = result['analysis']

    print(f"🎯 التنبؤ:")
    print(f"   الشخصية الأولى: {pred['top_character']}")
    print(f"   الثقة: {pred['confidence_percent']} ({pred['confidence_label']})")
    print(f"   نوع النمط: {pred['pattern_type'].upper()}")
    print(f"   اليقين: {pred['certainty_level']}")
    print(f"   أفضل 3: {', '.join(pred['top_3_characters'])}")

    print(f"\n🔍 التحليل:")
    print(f"   قوة الأدلة: {analysis['evidence_strength'].upper()}")
    print(f"   وضوح النمط: {analysis['pattern_clarity']:.2f}")
    print(f"   درجة الغموض: {analysis['ambiguity_score']:.2f}")

    # التحقق من الدقة (للعرض التوضيحي فقط)
    if pred['top_character'] == true_dominant:
        print(f"\n✅ صحيح: تم التنبؤ بالجزء المهيمن الحقيقي!")
    else:
        print(f"\n⚠️  مختلف: تم التنبؤ بـ {pred['top_character']} بدلاً من {true_dominant}")
        if true_dominant in pred['top_3_characters']:
            print(f"   (لكن {true_dominant} موجود في أفضل 3)")

    # عرض الاحتمالات الأولية لأفضل 5
    print(f"\n📊 أفضل 5 احتمالات:")
    all_probs = result['raw_data']['all_probabilities']
    sorted_probs = sorted(all_probs.items(), key=lambda x: x[1], reverse=True)[:5]
    for char, prob in sorted_probs:
        print(f"   {char:20}: {prob:.1%}")

except Exception as e:
    print(f"❌ خطأ في العرض التوضيحي: {e}")

print("\n" + "="*80)
print("✅ النموذج الإنتاجي الثابت بالعربية مكتمل")
print("="*80)
print("\nالتحسينات الرئيسية المنفذة:")
print("✅ التعامل الصحيح للفئات لجميع الـ 18 شخصية")
print("✅ معايرة الثقة الواقعية حسب نوع النمط")
print("✅ تحليل القوة القائم على الأدلة")
print("✅ نظام تدقيق الإجابات والتحذيرات")
print("✅ تصنيف الأنماط النفسية")
print("✅ جاهز للإنتاج مع التعامل الصحيح للأخطاء")
print("\n🚀 جاهز للنشر!")

🚀 النموذج الجاهز للاستخدام للتنبؤ بالشخصيات (العربية)

1. تحميل مجموعة البيانات العربية...
   ✅ تم تحميل مجموعة البيانات: 100,000 عينة

2. إنشاء الميزات المتقدمة...

3. تدريب النموذج الإنتاجي الثابت مع التعامل الصحيح للفئات...

🚀 تدريب النموذج الإنتاجي الثابت بالعربية
   إنشاء الميزات...
   تم العثور على 18 فئة فريدة في البيانات
   تصنيف أنواع الأنماط...
   توزيع أنواع الأنماط:
mixed         44274
ambiguous     28511
very_clear    21628
clear          5587
Name: count, dtype: int64
   تدريب النموذج الرئيسي...
   تدريب نموذج التجميع...
   دقة النموذج الرئيسي: 95.80%
   تدريب نماذج الاحتياط المحددة بالأنماط...
     تدريب نموذج mixed (44,274 عينة)...
       دقة الاختبار: 97.00%
     تدريب نموذج ambiguous (28,511 عينة)...
       دقة الاختبار: 95.00%
     تدريب نموذج very_clear (21,628 عينة)...
       دقة الاختبار: 91.00%
     تدريب نموذج clear (5,587 عينة)...
       دقة الاختبار: 88.00%
   ✅ تم تدريب جميع النماذج بنجاح
   ✅ تم حفظ النموذج في fixed_production_character_predictor_ar.pkl

🧪 ا

In [ ]:
# ==================== 13. حالات اختبار محددة مسبقاً للنموذج العربي ====================
print("\n" + "="*80)
print("🧪 10 حالات اختبار محددة مسبقاً للنموذج العربي")
print("="*80)

predefined_test_cases_ar = [
    {
        "id": 1,
        "name": "الكمالي الواضح جداً (نموذجي)",
        "description": "شخصية كمالية واضحة مع إجابات متسقة، متوقع ثقة عالية جداً",
        "dominant_character": "الكمالي",
        "answers": {
            "Q1": "0", "Q2": "81-100%", "Q3": "0", "Q4": "0-20%",
            "Q5": "0", "Q6": "0", "Q7": "0", "Q8": "0-20%",
            "Q9": "3", "Q10": "1", "Q11": "0", "Q12": "0", "Q13": "2"
        }
    },
    {
        "id": 2,
        "name": "الناقد الداخلي + الخجول (مختلط قوي)",
        "description": "مزيج قوي من الناقد الداخلي والجزء الخجول، ثقة متوسطة",
        "dominant_character": "الناقد الداخلي",
        "answers": {
            "Q1": "0,4", "Q2": "81-100%", "Q3": "3,5", "Q4": "21-50%",
            "Q5": "0,4", "Q6": "0,5", "Q7": "5", "Q8": "0-20%",
            "Q9": "2", "Q10": "5", "Q11": "0", "Q12": "0,4", "Q13": "5"
        }
    },
    {
        "id": 3,
        "name": "المُرضي للناس + المعتمد (اجتماعي قوي)",
        "description": "تركيز قوي على العلاقات الاجتماعية والاعتماد على الآخرين",
        "dominant_character": "المُرضي للناس",
        "answers": {
            "Q1": "2", "Q2": "51-80%", "Q3": "3", "Q4": "51-80%",
            "Q5": "2", "Q6": "2,3", "Q7": "3", "Q8": "21-50%",
            "Q9": "0,5", "Q10": "0", "Q11": "4", "Q12": "4", "Q13": "3,6"
        }
    },
    {
        "id": 4,
        "name": "المتحكّم + الكمالي (تحكم كامل)",
        "description": "شخصية متحكمة وكمالية، ثقة عالية في التحكم والكمالية",
        "dominant_character": "المتحكّم",
        "answers": {
            "Q1": "0", "Q2": "81-100%", "Q3": "0", "Q4": "0-20%",
            "Q5": "0,2", "Q6": "0", "Q7": "0", "Q8": "0-20%",
            "Q9": "3", "Q10": "1", "Q11": "0,2", "Q12": "0", "Q13": "2"
        }
    },
    {
        "id": 5,
        "name": "الجزء الوحيد + المهمَل (وحدة وإهمال)",
        "description": "مشاعر عميقة بالوحدة والإهمال، أنماط واضحة",
        "dominant_character": "الجزء الوحيد",
        "answers": {
            "Q1": "5", "Q2": "21-50%", "Q3": "2,4", "Q4": "81-100%",
            "Q5": "1", "Q6": "3", "Q7": "2", "Q8": "51-80%",
            "Q9": "1,4", "Q10": "4", "Q11": "1,3", "Q12": "3", "Q13": "0"
        }
    },
    {
        "id": 6,
        "name": "المؤجّل + اللاعب المفرط (هروب وتأجيل)",
        "description": "نمط واضح من التأجيل والهروب عبر الألعاب والتسلية",
        "dominant_character": "المؤجِّل",
        "answers": {
            "Q1": "3", "Q2": "0-20%", "Q3": "1", "Q4": "21-50%",
            "Q5": "3", "Q6": "4", "Q7": "1,4", "Q8": "81-100%",
            "Q9": "4", "Q10": "4", "Q11": "5", "Q12": "5", "Q13": "4"
        }
    },
    {
        "id": 7,
        "name": "مدمن الشغل + الكمالي (عمل وكمالية)",
        "description": "تركيز على العمل والإنتاجية مع صفات كمالية",
        "dominant_character": "مدمن الشغل",
        "answers": {
            "Q1": "0,1", "Q2": "81-100%", "Q3": "0", "Q4": "0-20%",
            "Q5": "0", "Q6": "0", "Q7": "0", "Q8": "0-20%",
            "Q9": "3", "Q10": "1", "Q11": "0", "Q12": "0", "Q13": "2"
        }
    },
    {
        "id": 8,
        "name": "الجزء الحيران + المهمَل (حيرة وإهمال)",
        "description": "مشاعر الحيرة والانفصال عن المشاعر",
        "dominant_character": "الجزء الحيران",
        "answers": {
            "Q1": "5", "Q2": "21-50%", "Q3": "4", "Q4": "51-80%",
            "Q5": "5", "Q6": "3", "Q7": "2", "Q8": "21-50%",
            "Q9": "4", "Q10": "4", "Q11": "3,5", "Q12": "5", "Q13": "7"
        }
    },
    {
        "id": 9,
        "name": "الطفل الجريح + الخجول (جراح وخزي)",
        "description": "مشاعر الجرح والخزي من الطفولة، حساسية عالية",
        "dominant_character": "الطفل الجريح",
        "answers": {
            "Q1": "2,5", "Q2": "21-50%", "Q3": "2,5", "Q4": "51-80%",
            "Q5": "4,5", "Q6": "5", "Q7": "2,5", "Q8": "51-80%",
            "Q9": "4", "Q10": "5", "Q11": "0,1", "Q12": "1", "Q13": "0,6"
        }
    },
    {
        "id": 10,
        "name": "الجزء المرهَق + الخايف (إرهاق وخوف)",
        "description": "مشاعر الإرهاق والخوف مع استجابات عاطفية قوية",
        "dominant_character": "الجزء المرهَق",
        "answers": {
            "Q1": "4,5", "Q2": "21-50%", "Q3": "2", "Q4": "51-80%",
            "Q5": "4", "Q6": "1,4", "Q7": "2", "Q8": "51-80%",
            "Q9": "1,5", "Q10": "5", "Q11": "2", "Q12": "4", "Q13": "0"
        }
    }
]

def run_predefined_tests(predictor, test_cases):
    """تشغيل حالات الاختبار المحددة مسبقاً"""
    results = []
    detailed_report = []

    print(f"\n{'='*80}")
    print(f"🚀 تشغيل {len(test_cases)} حالة اختبار محددة مسبقاً")
    print(f"{'='*80}")

    for test_case in test_cases:
        print(f"\n🔬 اختبار {test_case['id']}: {test_case['name']}")
        print(f"   📝 {test_case['description']}")
        print(f"   🎯 المتوقع: {test_case['dominant_character']}")
        print("-" * 60)

        try:
            # التنبؤ
            result = predictor.predict_with_confidence(test_case['answers'])
            pred = result['prediction']
            analysis = result['analysis']
            warnings = result['warnings']

            # عرض النتائج
            print(f"   🎯 التنبؤ: {pred['top_character']}")
            print(f"   📊 الثقة: {pred['confidence_percent']} ({pred['confidence_label']})")
            print(f"   🏷️  نوع النمط: {pred['pattern_type']}")
            print(f"   💪 قوة الأدلة: {analysis['evidence_strength']}")

            # التحقق من الدقة
            correct_prediction = (pred['top_character'] == test_case['dominant_character'])
            prediction_in_top3 = (test_case['dominant_character'] in pred['top_3_characters'])

            if correct_prediction:
                print(f"   ✅ دقة: صحيح! (في المركز الأول)")
            elif prediction_in_top3:
                rank = pred['top_3_characters'].index(test_case['dominant_character']) + 1
                print(f"   ⚠️  دقة: جزئي (في المركز {rank})")
            else:
                print(f"   ❌ دقة: غير صحيح")

            # عرض أفضل 3
            print(f"   🥇 أفضل 3:")
            for i, char in enumerate(pred['top_3_characters'], 1):
                prob_key = f'top{i}_prob'
                prob = result['raw_data']['all_probabilities'].get(char, 0)
                print(f"      {i}. {char} ({prob:.1%})")

            # التحذيرات
            if warnings['has_warnings']:
                print(f"   ⚠️  تحذيرات:")
                for warning in warnings['validation_warnings']:
                    print(f"      • {warning}")

            # تحليل الواقعية النفسية
            print(f"   🧠 تحليل واقعية:")

            # نطاق الثقة المتوقع
            if pred['pattern_type'] in ['very_clear', 'clear']:
                expected_range = (0.75, 0.95)
            elif pred['pattern_type'] == 'mixed':
                expected_range = (0.50, 0.70)
            elif pred['pattern_type'] == 'ambiguous':
                expected_range = (0.30, 0.50)
            else:
                expected_range = (0.15, 0.35)

            confidence = pred['confidence']
            confidence_realistic = expected_range[0] <= confidence <= expected_range[1]

            if confidence_realistic:
                print(f"      ✅ الثقة واقعية للنمط {pred['pattern_type']}")
            else:
                print(f"      ⚠️  الثقة غير واقعية للنمط {pred['pattern_type']}")
                print(f"         المتوقع: {expected_range[0]:.1%}-{expected_range[1]:.1%}")
                print(f"         الفعلي: {confidence:.1%}")

            # جمع النتائج
            test_result = {
                'id': test_case['id'],
                'name': test_case['name'],
                'expected': test_case['dominant_character'],
                'predicted': pred['top_character'],
                'confidence': confidence,
                'pattern_type': pred['pattern_type'],
                'evidence_strength': analysis['evidence_strength'],
                'correct_prediction': correct_prediction,
                'in_top3': prediction_in_top3,
                'top3_rank': pred['top_3_characters'].index(test_case['dominant_character']) + 1
                           if prediction_in_top3 else None,
                'confidence_realistic': confidence_realistic,
                'has_warnings': warnings['has_warnings'],
                'warning_count': len(warnings['validation_warnings'])
            }

            results.append(test_result)

            # تقرير مفصل
            detailed_report.append({
                'test_id': test_case['id'],
                'test_name': test_case['name'],
                'expected': test_case['dominant_character'],
                'predicted': pred['top_character'],
                'top_3': pred['top_3_characters'],
                'confidence': f"{pred['confidence_percent']} ({pred['confidence_label']})",
                'pattern_type': pred['pattern_type'],
                'evidence_strength': analysis['evidence_strength'],
                'warnings': warnings['validation_warnings']
            })

        except Exception as e:
            print(f"   ❌ خطأ في الاختبار: {e}")
            results.append({
                'id': test_case['id'],
                'name': test_case['name'],
                'error': str(e)
            })

    return results, detailed_report

# تشغيل حالات الاختبار
test_results, detailed_report = run_predefined_tests(predictor_ar, predefined_test_cases_ar)

# ==================== 14. تحليل شامل لنتائج الاختبارات ====================
print(f"\n{'='*80}")
print(f"📊 تحليل شامل لنتائج الاختبارات المحددة مسبقاً")
print(f"{'='*80}")

if test_results:
    # الإحصائيات العامة
    successful_tests = [r for r in test_results if 'error' not in r]
    total_tests = len(test_results)
    successful_count = len(successful_tests)

    print(f"\n📈 الإحصائيات العامة:")
    print(f"   إجمالي الاختبارات: {total_tests}")
    print(f"   الاختبارات الناجحة: {successful_count}")

    if successful_count > 0:
        # دقة التنبؤ
        correct_predictions = sum(1 for r in successful_tests if r['correct_prediction'])
        in_top3_predictions = sum(1 for r in successful_tests if r['in_top3'])

        accuracy_rate = (correct_predictions / successful_count) * 100
        top3_accuracy_rate = (in_top3_predictions / successful_count) * 100

        print(f"\n🎯 دقة التنبؤ:")
        print(f"   ✅ في المركز الأول: {correct_predictions}/{successful_count} ({accuracy_rate:.1f}%)")
        print(f"   🥇 في أفضل 3: {in_top3_predictions}/{successful_count} ({top3_accuracy_rate:.1f}%)")

        # تحليل الثقة
        confidence_realistic_count = sum(1 for r in successful_tests if r.get('confidence_realistic', False))
        confidence_realistic_rate = (confidence_realistic_count / successful_count) * 100

        avg_confidence = np.mean([r['confidence'] for r in successful_tests]) * 100
        min_confidence = min([r['confidence'] for r in successful_tests]) * 100
        max_confidence = max([r['confidence'] for r in successful_tests]) * 100

        print(f"\n📊 تحليل الثقة:")
        print(f"   📈 متوسط الثقة: {avg_confidence:.1f}%")
        print(f"   📉 أقل ثقة: {min_confidence:.1f}%")
        print(f"   📈 أعلى ثقة: {max_confidence:.1f}%")
        print(f"   ✅ واقعية الثقة: {confidence_realistic_count}/{successful_count} ({confidence_realistic_rate:.1f}%)")

        # توزيع قوة الأدلة
        print(f"\n🔍 توزيع قوة الأدلة:")
        evidence_counts = Counter([r['evidence_strength'] for r in successful_tests])
        for strength, count in evidence_counts.items():
            percentage = (count / successful_count) * 100
            print(f"   {strength.upper():15}: {count} اختبار ({percentage:.1f}%)")

        # توزيع أنواع الأنماط
        print(f"\n🎭 توزيع أنواع الأنماط:")
        pattern_counts = Counter([r['pattern_type'] for r in successful_tests])
        for pattern, count in pattern_counts.items():
            percentage = (count / successful_count) * 100
            print(f"   {pattern.upper():15}: {count} اختبار ({percentage:.1f}%)")

        # الثقة حسب نوع النمط
        print(f"\n📈 متوسط الثقة حسب نوع النمط:")
        pattern_groups = {}
        for result in successful_tests:
            pattern = result['pattern_type']
            if pattern not in pattern_groups:
                pattern_groups[pattern] = []
            pattern_groups[pattern].append(result['confidence'] * 100)

        for pattern, confidences in pattern_groups.items():
            if confidences:
                avg = np.mean(confidences)
                min_val = min(confidences)
                max_val = max(confidences)
                print(f"   {pattern.upper():15}: {avg:.1f}% (المدى: {min_val:.1f}%-{max_val:.1f}%)")

        # التحذيرات
        tests_with_warnings = sum(1 for r in successful_tests if r['has_warnings'])
        total_warnings = sum(r['warning_count'] for r in successful_tests)

        print(f"\n⚠️  التحذيرات:")
        print(f"   اختبارات بها تحذيرات: {tests_with_warnings}/{successful_count}")
        print(f"   إجمالي التحذيرات: {total_warnings}")

        # تقييم النموذج
        print(f"\n🏆 تقييم النموذج:")

        if accuracy_rate >= 80 and confidence_realistic_rate >= 80:
            print("   ✅ ممتاز: النموذج يعمل بشكل ممتاز")
            print("   🎯 الدقة عالية والثقة واقعية")
        elif accuracy_rate >= 60 and confidence_realistic_rate >= 60:
            print("   ⚠️  جيد: النموذج يعمل بشكل جيد")
            print("   📈 يمكن تحسين الدقة والثقة")
        elif accuracy_rate >= 40:
            print("   ⚠️  مقبول: النموذج بحاجة لتحسين")
            print("   🔧 انخفاض في الدقة أو واقعية الثقة")
        else:
            print("   ❌ يحتاج تحسين كبير")
            print("   🛠️  انخفاض شديد في الدقة")

        # النتائج التفصيلية لكل اختبار
        print(f"\n📋 النتائج التفصيلية لكل اختبار:")
        print("-" * 100)
        print(f"{'ID':<3} {'الاختبار':<30} {'المتوقع':<15} {'التنبؤ':<15} {'الدقة':<10} {'الثقة':<15} {'النمط':<12}")
        print("-" * 100)

        for result in successful_tests:
            accuracy_symbol = "✅" if result['correct_prediction'] else "❌"
            accuracy_text = "صحيح" if result['correct_prediction'] else f"المركز {result['top3_rank']}" if result['in_top3'] else "غير صحيح"

            print(f"{result['id']:<3} {result['name'][:28]:<30} "
                  f"{result['expected'][:13]:<15} {result['predicted'][:13]:<15} "
                  f"{accuracy_symbol + accuracy_text:<10} {result['confidence']:.1%}:<15 "
                  f"{result['pattern_type']:<12}")

    else:
        print("❌ لا توجد نتائج ناجحة للتحليل")

    # توصيات للتحسين
    print(f"\n💡 توصيات للتحسين:")

    successful_with_results = [r for r in successful_tests if 'confidence' in r]
    if successful_with_results:
        avg_acc = accuracy_rate if 'accuracy_rate' in locals() else 0
        avg_conf_real = confidence_realistic_rate if 'confidence_realistic_rate' in locals() else 0

        if avg_acc < 70:
            print("   1. تحسين دقة التنبؤ للشخصيات المتعددة")
            print("   2. مراجعة توقيعات الشخصيات للأنماط المختلطة")

        if avg_conf_real < 70:
            print("   3. ضبط معلمات معايرة الثقة")
            print("   4. تحسين تصنيف الأنماط النفسية")

        if total_warnings > 0:
            print("   5. تحسين نظام تدقيق الإجابات للحد من التحذيرات الكاذبة")

        print("   6. جمع المزيد من البيانات للأنماط النادرة")
        print("   7. اختبار النموذج على مستخدمين حقيقيين لجمع الملاحظات")

else:
    print("❌ لا توجد نتائج للتحليل")

# ==================== 15. حفظ تقرير مفصل بالنتائج ====================
print(f"\n{'='*80}")
print(f"💾 حفظ التقارير والنتائج")
print(f"{'='*80}")

# حفظ التقرير التفصيلي
if detailed_report:
    report_df = pd.DataFrame(detailed_report)
    report_df.to_csv('test_results_detailed_report_ar.csv', index=False, encoding='utf-8-sig')
    print(f"✅ تم حفظ التقرير التفصيلي في: test_results_detailed_report_ar.csv")

# حفظ ملخص النتائج
if test_results:
    summary_data = []
    for result in test_results:
        if 'error' not in result:
            summary_data.append({
                'test_id': result['id'],
                'test_name': result['name'],
                'expected_character': result['expected'],
                'predicted_character': result['predicted'],
                'confidence': result['confidence'],
                'pattern_type': result['pattern_type'],
                'evidence_strength': result['evidence_strength'],
                'correct_prediction': result['correct_prediction'],
                'in_top3': result['in_top3'],
                'confidence_realistic': result['confidence_realistic'],
                'warning_count': result['warning_count']
            })

    if summary_data:
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv('test_results_summary_ar.csv', index=False, encoding='utf-8-sig')
        print(f"✅ تم حفظ ملخص النتائج في: test_results_summary_ar.csv")

# عرض إحصائيات سريعة
print(f"\n📊 الإحصائيات السريعة:")
print(f"   - إجمالي الاختبارات: {len(predefined_test_cases_ar)}")
print(f"   - تم تنفيذها بنجاح: {len([r for r in test_results if 'error' not in r])}")
print(f"   - مدة التدريب: حوالي 2-5 دقائق")
print(f"   - حجم النموذج: ~10-20 ميجابايت")
print(f"   - وقت التنبؤ: <1 ثانية لكل اختبار")

print(f"\n{'='*80}")
print(f"🎉 اختبار النموذج العربي اكتمل بنجاح!")
print(f"{'='*80}")


🧪 10 حالات اختبار محددة مسبقاً للنموذج العربي

🚀 تشغيل 10 حالة اختبار محددة مسبقاً

🔬 اختبار 1: الكمالي الواضح جداً (نموذجي)
   📝 شخصية كمالية واضحة مع إجابات متسقة، متوقع ثقة عالية جداً
   🎯 المتوقع: الكمالي
------------------------------------------------------------
   🎯 التنبؤ: الكمالي
   📊 الثقة: 85.0% (ثقة عالية)
   🏷️  نوع النمط: very_clear
   💪 قوة الأدلة: very_strong
   ✅ دقة: صحيح! (في المركز الأول)
   🥇 أفضل 3:
      1. الكمالي (44.0%)
      2. المتحكّم (22.8%)
      3. مدمن الشغل (14.7%)
   🧠 تحليل واقعية:
      ✅ الثقة واقعية للنمط very_clear

🔬 اختبار 2: الناقد الداخلي + الخجول (مختلط قوي)
   📝 مزيج قوي من الناقد الداخلي والجزء الخجول، ثقة متوسطة
   🎯 المتوقع: الناقد الداخلي
------------------------------------------------------------
   🎯 التنبؤ: الناقد الداخلي
   📊 الثقة: 85.0% (ثقة عالية)
   🏷️  نوع النمط: very_clear
   💪 قوة الأدلة: very_strong
   ✅ دقة: صحيح! (في المركز الأول)
   🥇 أفضل 3:
      1. الناقد الداخلي (41.3%)
      2. الجزء الخجول (37.9%)
      3. الجزء 